<a href="https://colab.research.google.com/github/vpxop111/scamrASKSHAK/blob/master/scamwebsite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install torch pandas scikit-learn

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Load your dataset
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/verified_online.csv'
df = pd.read_csv(data_path, encoding='utf-8')


# Ensure the DataFrame contains the columns 'url' and 'label'
assert 'url' in df.columns and 'label' in df.columns, "CSV file must contain 'url' and 'label' columns."

# Function to preprocess URLs: Tokenization and padding
def preprocess_url(url):
    url = url.lower()
    url = re.sub(r'https?://', '', url)  # Remove protocol
    url = re.sub(r'www\.', '', url)       # Remove 'www'
    return ' '.join(list(url))            # Tokenize by characters

class URLDataset(Dataset):
    def __init__(self, urls, labels, max_length):
        self.urls = urls
        self.labels = labels
        self.max_length = max_length
        self.tokenizer = LabelEncoder()
        self.tokenizer.fit([char for url in urls for char in preprocess_url(url)])

    def __len__(self):
        return len(self.urls)

    def __getitem__(self, idx):
        url = preprocess_url(self.urls[idx])
        url_tokens = self.tokenizer.transform(list(url))
        url_padded = np.pad(url_tokens, (0, self.max_length - len(url_tokens)), mode='constant')
        return torch.tensor(url_padded, dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

# Parameters
max_length = 50
batch_size = 2
num_classes = 2
embedding_dim = 10
hidden_dim = 20
num_layers = 1
learning_rate = 0.001
num_epochs = 10

# Data preparation
X = df['url'].values
y = df['label'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = URLDataset(X_train, y_train, max_length)
val_dataset = URLDataset(X_val, y_val, max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_classes):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        h0 = torch.zeros(self.rnn.num_layers, x.size(0), hidden_dim).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# Initialize the model, loss function, and optimizer
vocab_size = len(train_dataset.tokenizer.classes_)
model = RNNModel(vocab_size, embedding_dim, hidden_dim, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for urls, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(urls)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}')

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for urls, labels in val_loader:
            outputs = model(urls)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'Validation Accuracy: {accuracy * 100:.2f}%')

# Train and evaluate the model
train_model(model, train_loader, criterion, optimizer, num_epochs)
evaluate_model(model, val_loader)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


AssertionError: CSV file must contain 'url' and 'label' columns.

In [ ]:
pip install datallm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming you're running this in Google Colab
from google.colab import drive
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/dataset_phishing.csv'
df = pd.read_csv(data_path, encoding='utf-8')

# Load and preprocess data
df.dropna(inplace=True)

# Features and target
features = [
    'url', 'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq',
    'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma', 'nb_semicolumn',
    'nb_dollar', 'nb_space', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url',
    'ratio_digits_host', 'punycode', 'port', 'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains',
    'prefix_suffix', 'random_domain', 'shortening_service', 'path_extension', 'nb_redirection', 'nb_external_redirection',
    'length_words_raw', 'char_repeat', 'shortest_words_raw', 'shortest_word_host', 'shortest_word_path',
    'longest_words_raw', 'longest_word_host', 'longest_word_path', 'avg_words_raw', 'avg_word_host', 'avg_word_path',
    'phish_hints', 'domain_in_brand', 'brand_in_subdomain', 'brand_in_path', 'suspecious_tld',
    'nb_hyperlinks', 'ratio_intHyperlinks', 'ratio_extHyperlinks', 'ratio_nullHyperlinks', 'nb_extCSS',
    'ratio_intRedirection', 'ratio_extRedirection', 'ratio_intErrors', 'ratio_extErrors', 'login_form',
    'external_favicon', 'links_in_tags', 'submit_email', 'ratio_intMedia', 'ratio_extMedia', 'sfh', 'iframe',
    'popup_window', 'safe_anchor', 'onmouseover', 'right_clic', 'empty_title', 'domain_in_title',
    'domain_with_copyright', 'whois_registered_domain', 'domain_registration_length', 'domain_age'
]

df['status'] = df['status'].map({'phishing': 1, 'legitimate': 0})

X = df[features]
y = df['status']

# Convert 'url' column to numerical
X['url'] = pd.factorize(X['url'])[0]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Define the RNN model
class PhishingRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2):
        super(PhishingRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# Hyperparameters
input_size = X_train_tensor.shape[1]
hidden_size = 128
output_size = 2
num_layers = 1
num_epochs = 100
batch_size = 64
learning_rate = 0.005

# Initialize the model, loss function, and optimizer
model = PhishingRNN(input_size, hidden_size, output_size, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train_tensor.unsqueeze(1))
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluating the model
model.eval()
with torch.no_grad():
    y_pred_train = model(X_train_tensor.unsqueeze(1)).argmax(dim=1)
    y_pred_test = model(X_test_tensor.unsqueeze(1)).argmax(dim=1)

    train_accuracy = accuracy_score(y_train, y_pred_train.numpy())
    test_accuracy = accuracy_score(y_test, y_pred_test.numpy())

    print(f'Train Accuracy: {train_accuracy:.4f}')
    print(f'Test Accuracy: {test_accuracy:.4f}')
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test.numpy()))
    print("Classification Report:\n", classification_report(y_test, y_pred_test.numpy()))

# Function to test a new URL
def test_new_url(url_features):
    assert len(url_features) == len(features), f"Expected {len(features)} features, but got {len(url_features)}."

    # Convert url to numerical value
    url_features[0] = pd.factorize([url_features[0]])[0][0]

    model.eval()
    url_features_scaled = scaler.transform([url_features])
    url_tensor = torch.tensor(url_features_scaled, dtype=torch.float32).unsqueeze(1)
    with torch.no_grad():
        prediction = model(url_tensor).argmax(dim=1).item()
    return 'Phishing' if prediction == 1 else 'Legitimate'

# Example usage
new_url_features = [
    'https://www.youtube.com', 23, 15, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0.0, 0.0, 0, 443, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 7, 0, 3, 7, 3, 7, 7, 7, 3.29, 7.0, 3.0, 0, 0, 0, 0, 0, 10, 1.0, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 1.0, 0.0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 365, 5844
]

# Test the new URL
result = test_new_url(new_url_features)
print(f'The new URL is classified as: {result}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-16-cad56c18bccb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['url'] = pd.factorize(X['url'])[0]


Epoch [10/100], Loss: 0.2955
Epoch [20/100], Loss: 0.2404
Epoch [30/100], Loss: 0.2186
Epoch [40/100], Loss: 0.2021
Epoch [50/100], Loss: 0.1836
Epoch [60/100], Loss: 0.1635
Epoch [70/100], Loss: 0.1419
Epoch [80/100], Loss: 0.1198
Epoch [90/100], Loss: 0.0991
Epoch [100/100], Loss: 0.0838
Train Accuracy: 0.9718
Test Accuracy: 0.9405
Confusion Matrix:
 [[1347   75]
 [  95 1341]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.95      0.94      1422
           1       0.95      0.93      0.94      1436

    accuracy                           0.94      2858
   macro avg       0.94      0.94      0.94      2858
weighted avg       0.94      0.94      0.94      2858

The new URL is classified as: Phishing


<ipython-input-16-cad56c18bccb>:121: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  url_features[0] = pd.factorize([url_features[0]])[0][0]
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
pip install --upgrade python-whois

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import re
import whois
import tldextract
import datetime
import socket

def is_valid_ip(ip):
    """Check if the hostname is an IP address"""
    try:
        socket.inet_aton(ip)
        return True
    except socket.error:
        return False

def extract_features(url):
    features = {}

    # Basic URL parsing
    parsed_url = urlparse(url)
    domain_info = tldextract.extract(url)

    # Having IP Address
    features['having_IP_Address'] = -1 if is_valid_ip(parsed_url.hostname) else 1

    # URL Length
    url_len = len(url)
    features['URL_Length'] = 1 if url_len < 54 else (0 if 54 <= url_len <= 75 else -1)

    # Shortening Service
    shortening_services = ['bit.ly', 'tinyurl.com', 'goo.gl']
    features['Shortining_Service'] = -1 if any(service in url for service in shortening_services) else 1

    # Having '@' Symbol
    features['having_At_Symbol'] = -1 if '@' in url else 1

    # Double Slash Redirecting
    features['double_slash_redirecting'] = -1 if url.count('//') > 1 else 1

    # Prefix-Suffix in Domain
    features['Prefix_Suffix'] = -1 if '-' in parsed_url.netloc else 1

    # Having Sub Domain
    subdomains = domain_info.subdomain.split('.')
    if len(subdomains) == 0:
        features['having_Sub_Domain'] = -1
    elif len(subdomains) == 1:
        features['having_Sub_Domain'] = 0
    else:
        features['having_Sub_Domain'] = 1

    # SSLfinal State (simplified check)
    features['SSLfinal_State'] = 1 if parsed_url.scheme == 'https' else -1

    # Domain Registration Length (simplified for example)
    try:
        whois_info = whois.whois(parsed_url.hostname)
        if whois_info.expiration_date and whois_info.creation_date:
            if isinstance(whois_info.expiration_date, list):
                expiration_date = whois_info.expiration_date[0]
            else:
                expiration_date = whois_info.expiration_date
            registration_length = (expiration_date - whois_info.creation_date).days
            features['Domain_registeration_length'] = 1 if registration_length > 365 else -1
        else:
            features['Domain_registeration_length'] = -1
    except:
        features['Domain_registeration_length'] = -1

    # Favicon (simplified for example)
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.content, 'html.parser')
        favicon_tag = soup.find('link', rel='shortcut icon')
        features['Favicon'] = -1 if favicon_tag and parsed_url.netloc not in favicon_tag.get('href', '') else 1
    except:
        features['Favicon'] = -1

    # Port (simplified check)
    features['port'] = -1 if parsed_url.port and parsed_url.port != 80 and parsed_url.port != 443 else 1

    # HTTPS Token in URL
    features['HTTPS_token'] = -1 if 'https-' in parsed_url.hostname else 1

    # Request URL (simplified for example)
    try:
        external_resources = [tag.get('src') for tag in soup.find_all('script', src=True)] + \
                             [tag.get('href') for tag in soup.find_all('link', href=True)]
        features['Request_URL'] = 1 if all(parsed_url.netloc in resource for resource in external_resources) else -1
    except:
        features['Request_URL'] = -1

    # URL of Anchor
    try:
        links = soup.find_all('a', href=True)
        total_links = len(links)
        if total_links > 0:
            external_links = sum([1 for link in links if parsed_url.netloc not in link['href']])
            features['URL_of_Anchor'] = -1 if external_links > (total_links * 0.67) else (0 if (total_links * 0.31) <= external_links <= (total_links * 0.67) else 1)
        else:
            features['URL_of_Anchor'] = 1
    except:
        features['URL_of_Anchor'] = 1

    # Links in Tags (script, link)
    try:
        script_links = len(soup.find_all('script', src=True))
        link_links = len(soup.find_all('link', href=True))
        total_links = script_links + link_links
        features['Links_in_tags'] = 1 if total_links == 0 else (0 if total_links < 2 else -1)
    except:
        features['Links_in_tags'] = -1

    # Server Form Handler (SFH)
    try:
        forms = soup.find_all('form', action=True)
        if len(forms) > 0:
            action_values = [form['action'] for form in forms]
            if any('mailto:' in action for action in action_values):
                features['SFH'] = -1
            elif any('https' in action or 'http' in action for action in action_values):
                features['SFH'] = 1
            else:
                features['SFH'] = 0
        else:
            features['SFH'] = 1
    except:
        features['SFH'] = -1

    # Submitting to Email
    features['Submitting_to_email'] = -1 if any('mailto:' in form['action'] for form in soup.find_all('form', action=True)) else 1

    # Abnormal URL
    features['Abnormal_URL'] = 1 if domain_info.domain in parsed_url.hostname else -1

    # Redirect (simplified for example)
    try:
        features['Redirect'] = 1 if len(response.history) <= 1 else 0
    except:
        features['Redirect'] = 0

    # On Mouseover
    features['on_mouseover'] = -1 if bool(soup.find_all(attrs={"onmouseover": True})) else 1

    # RightClick
    features['RightClick'] = -1 if bool(soup.find_all(attrs={"oncontextmenu": True})) else 1

    # popUpWindow (simplified for example)
    features['popUpWidnow'] = -1 if bool(soup.find('script', text=re.compile('window.open'))) else 1

    # Iframe
    features['Iframe'] = -1 if bool(soup.find('iframe')) else 1

    # Age of Domain
    try:
        features['age_of_domain'] = 1 if (datetime.datetime.now() - whois_info.creation_date).days > 180 else -1
    except:
        features['age_of_domain'] = -1

    # DNS Record
    try:
        features['DNSRecord'] = 1 if bool(whois_info.domain_name) else -1
    except:
        features['DNSRecord'] = -1

    # Web Traffic (placeholder, requires external data)
    features['web_traffic'] = 0  # Placeholder

    # PageRank (placeholder, requires external data)
    features['Page_Rank'] = 0  # Placeholder

    # Google Index (simplified)
    try:
        response = requests.get(f'https://www.google.com/search?q=site:{parsed_url.hostname}')
        features['Google_Index'] = 1 if 'did not match any documents' not in response.text else -1
    except:
        features['Google_Index'] = -1

    # Links Pointing to Page (simplified)
    try:
        features['Links_pointing_to_page'] = -1 if len(links) == 0 else (0 if len(links) <= 2 else 1)
    except:
        features['Links_pointing_to_page'] = 0

    # Statistical Report (placeholder)
    features['Statistical_report'] = 0  # Placeholder

    # Print all features
    for key, value in features.items():
        print(f"{key}: {value}")

    return features

# Example usage
url = "http://youtube.com"
extract_features(url)

<ipython-input-41-384d3981e745>:152: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  features['popUpWidnow'] = -1 if bool(soup.find('script', text=re.compile('window.open'))) else 1


having_IP_Address: 1
URL_Length: 1
Shortining_Service: 1
having_At_Symbol: 1
double_slash_redirecting: 1
Prefix_Suffix: 1
having_Sub_Domain: 0
SSLfinal_State: -1
Domain_registeration_length: -1
Favicon: 1
port: 1
HTTPS_token: 1
Request_URL: -1
URL_of_Anchor: 0
Links_in_tags: -1
SFH: 1
Submitting_to_email: 1
Abnormal_URL: 1
Redirect: 0
on_mouseover: 1
RightClick: 1
popUpWidnow: 1
Iframe: -1
age_of_domain: -1
DNSRecord: 1
web_traffic: 0
Page_Rank: 0
Google_Index: 1
Links_pointing_to_page: 1
Statistical_report: 0


{'having_IP_Address': 1,
 'URL_Length': 1,
 'Shortining_Service': 1,
 'having_At_Symbol': 1,
 'double_slash_redirecting': 1,
 'Prefix_Suffix': 1,
 'having_Sub_Domain': 0,
 'SSLfinal_State': -1,
 'Domain_registeration_length': -1,
 'Favicon': 1,
 'port': 1,
 'HTTPS_token': 1,
 'Request_URL': -1,
 'URL_of_Anchor': 0,
 'Links_in_tags': -1,
 'SFH': 1,
 'Submitting_to_email': 1,
 'Abnormal_URL': 1,
 'Redirect': 0,
 'on_mouseover': 1,
 'RightClick': 1,
 'popUpWidnow': 1,
 'Iframe': -1,
 'age_of_domain': -1,
 'DNSRecord': 1,
 'web_traffic': 0,
 'Page_Rank': 0,
 'Google_Index': 1,
 'Links_pointing_to_page': 1,
 'Statistical_report': 0}

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

drive.mount('/content/drive')

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/dataset_phishing.csv')
numerical_cols = df.select_dtypes(include=['float', 'int']).columns
X = df[numerical_cols].values

# Split the data into features and labels
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Normalize the data (if necessary)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Create DataLoader for batching
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)

        out, _ = self.rnn(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# Define the model
input_size = X.shape[1]
hidden_size = 128
num_layers = 2
output_size = 1  # Binary classification

model = RNNModel(input_size, hidden_size, num_layers, output_size)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()  # Since it's a binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
model.train()

for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs.squeeze(), y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


# Evaluation on test data
model.eval()
y_pred = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        predicted = torch.round(torch.sigmoid(outputs.squeeze()))
        y_pred.extend(predicted.cpu().numpy())

# Convert predictions to numpy array
y_pred = np.array(y_pred)

# Classification report
print(classification_report(y_test.numpy(), y_pred))

# Confusion matrix
conf_mat = confusion_matrix(y_test.numpy(), y_pred)
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ValueError: could not convert string to float: 'http://www.crestonwood.com/router.php'

In [ ]:
pip install numpy requests beautifulsoup4 python-whois tldextract


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.7 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import re
import whois
import tldextract
import datetime

# Mount your Google Drive
# drive.mount('/content/drive')

# Load and preprocess data
data_path = '/content/drive/MyDrive/dataset_phishing.csv'
df = pd.read_csv(data_path, encoding='utf-8')
df.dropna(inplace=True)

# Feature extraction function
def extract_features(url):
    features = {}
    parsed_url = urlparse(url)
    domain_info = tldextract.extract(url)

    # Basic URL Length Features
    features['length_url'] = len(url)
    features['length_hostname'] = len(parsed_url.hostname) if parsed_url.hostname else 0

    # Token Counts
    features['nb_dots'] = url.count('.')
    features['nb_hyphens'] = url.count('-')
    features['nb_at'] = url.count('@')
    features['nb_qm'] = url.count('?')
    features['nb_and'] = url.count('&')
    features['nb_or'] = url.count('|')
    features['nb_eq'] = url.count('=')
    features['nb_underscore'] = url.count('_')
    features['nb_tilde'] = url.count('~')
    features['nb_percent'] = url.count('%')
    features['nb_slash'] = url.count('/')
    features['nb_star'] = url.count('*')
    features['nb_colon'] = url.count(':')
    features['nb_comma'] = url.count(',')
    features['nb_semicolumn'] = url.count(';')
    features['nb_dollar'] = url.count('$')
    features['nb_space'] = url.count(' ')
    features['nb_www'] = url.count('www')
    features['nb_com'] = url.count('.com')
    features['nb_dslash'] = url.count('//')

    # Path Features
    features['http_in_path'] = 'http' in parsed_url.path
    features['https_token'] = 'https' in url
    features['ratio_digits_url'] = sum(c.isdigit() for c in url) / len(url) if len(url) > 0 else 0

    # Domain Features
    features['punycode'] = parsed_url.hostname.encode('idna').decode('utf-8') != parsed_url.hostname
    features['shortening_service'] = any(service in url for service in ['bit.ly', 'tinyurl.com', 'goo.gl'])
    features['port'] = parsed_url.port if parsed_url.port else -1
    features['tld_in_path'] = any(tld in parsed_url.path for tld in ['.com', '.net', '.org'])
    features['tld_in_subdomain'] = domain_info.suffix in parsed_url.hostname
    features['path_extension'] = any(ext in parsed_url.path for ext in ['.php', '.asp', '.aspx', '.jsp'])
    features['prefix_suffix'] = any(part in url for part in ['.htm', '.html'])
    features['random_domain'] = any(sub in parsed_url.hostname for sub in ['random', '1234', 'xyz'])

    # Fetch Page Content for More Features
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Count Links in the Page
        links = soup.find_all('a')
        features['nb_hyperlinks'] = len(links)
        features['ratio_extHyperlinks'] = len([link for link in links if link.get('href', '').startswith('http')]) / len(links) if len(links) > 0 else 0
        features['ratio_intHyperlinks'] = len([link for link in links if link.get('href', '') and not link.get('href', '').startswith('http')]) / len(links) if len(links) > 0 else 0
        features['ratio_nullHyperlinks'] = len([link for link in links if not link.get('href')]) / len(links) if len(links) > 0 else 0

        # Check for Login Form
        features['login_form'] = int(bool(soup.find('form') and (soup.find('input', {'type': 'password'}) or soup.find('input', {'type': 'text'}))))

        # Check for External CSS
        css_links = soup.find_all('link', {'rel': 'stylesheet'})
        features['nb_extCSS'] = len(css_links)

        # External Links and Media
        media_links = soup.find_all(['img', 'video', 'audio'])
        features['ratio_extMedia'] = len(media_links) / len(links) if len(links) > 0 else 0

        # Check for Popups and Iframes
        features['iframe'] = int(bool(soup.find('iframe')))
        features['popup_window'] = int(bool(soup.find('script', text=re.compile('window.open'))))

        # Additional Features
        features['sfh'] = int(bool(soup.find('form', action=lambda x: x and x.startswith('http'))))
        features['safe_anchor'] = int(all(link.get('rel') != 'nofollow' for link in links))
        features['onmouseover'] = int(bool(soup.find_all(attrs={"onmouseover": True})))
        features['right_clic'] = int(bool(soup.find_all(attrs={"oncontextmenu": True})))
        features['empty_title'] = int(bool(soup.find('title') and not soup.find('title').text.strip()))
        features['domain_in_title'] = int(domain_info.domain in (soup.find('title').text if soup.find('title') else ''))
        features['domain_with_copyright'] = int('copyright' in (soup.find('body').text if soup.find('body') else '').lower())

    except Exception as e:
        print(f"Error fetching URL content: {e}")
        features['nb_hyperlinks'] = 0
        features['ratio_extHyperlinks'] = 0
        features['ratio_intHyperlinks'] = 0
        features['ratio_nullHyperlinks'] = 0
        features['nb_extCSS'] = 0
        features['ratio_extMedia'] = 0
        features['iframe'] = 0
        features['popup_window'] = 0
        features['sfh'] = 0
        features['safe_anchor'] = 0
        features['onmouseover'] = 0
        features['right_clic'] = 0
        features['empty_title'] = 0
        features['domain_in_title'] = 0
        features['domain_with_copyright'] = 0

    # WHOIS Registration Features
    try:
        whois_info = whois.whois(parsed_url.hostname)
        features['whois_registered_domain'] = int(bool(whois_info.domain_name))
        features['domain_registration_length'] = whois_info.domain_age if whois_info.domain_age else -1
        features['domain_age'] = (datetime.datetime.now() - whois_info.creation_date).days if whois_info.creation_date else -1
    except Exception as e:
        print(f"Error fetching WHOIS info: {e}")
        features['whois_registered_domain'] = 0
        features['domain_registration_length'] = -1
        features['domain_age'] = -1

    return features

# Map the 'status' column to numeric values before splitting the data
df['status'] = df['status'].map({'legitimate': 0, 'phishing': 1})

# Feature and target selection
features = [
    'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq',
    'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma', 'nb_semicolumn',
    'nb_dollar', 'nb_space', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url',
    'ratio_digits_host', 'punycode', 'port', 'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains',
    'prefix_suffix', 'random_domain', 'shortening_service', 'path_extension', 'nb_redirection', 'nb_external_redirection',
    'length_words_raw', 'char_repeat', 'shortest_words_raw', 'shortest_word_host', 'shortest_word_path',
    'longest_words_raw', 'longest_word_host', 'longest_word_path', 'avg_words_raw', 'avg_word_host', 'avg_word_path',
    'phish_hints', 'domain_in_brand', 'brand_in_subdomain', 'brand_in_path', 'suspecious_tld', 'statistical_report',
    'nb_hyperlinks', 'ratio_intHyperlinks', 'ratio_extHyperlinks', 'ratio_nullHyperlinks', 'nb_extCSS',
    'ratio_intRedirection', 'ratio_extRedirection', 'ratio_intErrors', 'ratio_extErrors', 'login_form',
    'external_favicon', 'links_in_tags', 'submit_email', 'ratio_intMedia', 'ratio_extMedia', 'sfh', 'iframe',
    'popup_window', 'safe_anchor', 'onmouseover', 'right_clic', 'empty_title', 'domain_in_title',
    'domain_with_copyright', 'whois_registered_domain', 'domain_registration_length', 'domain_age'
]

X = df[features].values
y = df['status'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 128
num_layers = 2
output_size = 1
num_epochs = 100
learning_rate = 0.005

# Initialize the model, loss function, and optimizer
model = RNNModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train.unsqueeze(1))
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    y_pred_train = model(X_train.unsqueeze(1)).round()
    y_pred_test = model(X_test.unsqueeze(1)).round()

    train_accuracy = accuracy_score(y_train, y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    print(f'Training Accuracy: {train_accuracy:.4f}')
    print(f'Test Accuracy: {test_accuracy:.4f}')

    print('Classification Report:')
    print(classification_report(y_test, y_pred_test))

    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred_test))

# Predicting new URL
new_url = 'https://www.youtube.com'
new_features = extract_features(new_url)
new_features = pd.DataFrame([new_features])
new_features = scaler.transform(new_features)
new_features_tensor = torch.tensor(new_features, dtype=torch.float32).unsqueeze(1)
model.eval()
with torch.no_grad():
    new_prediction = model(new_features_tensor).round()
    print(f'Prediction for {new_url}: {"Phishing" if new_prediction.item() == 1 else "Legitimate"}')

Epoch [10/100], Loss: 0.4442
Epoch [20/100], Loss: 0.3165
Epoch [30/100], Loss: 0.2697
Epoch [40/100], Loss: 0.2510
Epoch [50/100], Loss: 0.2397
Epoch [60/100], Loss: 0.2323
Epoch [70/100], Loss: 0.2270
Epoch [80/100], Loss: 0.2222
Epoch [90/100], Loss: 0.2173
Epoch [100/100], Loss: 0.2118
Training Accuracy: 0.9170
Test Accuracy: 0.9147
Classification Report:
              precision    recall  f1-score   support

         0.0       0.91      0.92      0.92      1157
         1.0       0.92      0.90      0.91      1129

    accuracy                           0.91      2286
   macro avg       0.91      0.91      0.91      2286
weighted avg       0.91      0.91      0.91      2286

Confusion Matrix:
[[1070   87]
 [ 108 1021]]


<ipython-input-10-1c737c92aa6b>:97: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  features['popup_window'] = int(bool(soup.find('script', text=re.compile('window.open'))))


Error fetching WHOIS info: unsupported operand type(s) for -: 'datetime.datetime' and 'list'


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


ValueError: X has 52 features, but StandardScaler is expecting 83 features as input.

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import re
import tldextract
import whois
import datetime

# Feature extraction function
def extract_features(url):
    features = {}
    parsed_url = urlparse(url)
    domain_info = tldextract.extract(url)

    # Basic URL Length Features
    features['length_url'] = len(url)
    features['length_hostname'] = len(parsed_url.hostname) if parsed_url.hostname else 0

    # Token Counts
    features['nb_dots'] = url.count('.')
    features['nb_hyphens'] = url.count('-')
    features['nb_at'] = url.count('@')
    features['nb_qm'] = url.count('?')
    features['nb_and'] = url.count('&')
    features['nb_or'] = url.count('|')
    features['nb_eq'] = url.count('=')
    features['nb_underscore'] = url.count('_')
    features['nb_tilde'] = url.count('~')
    features['nb_percent'] = url.count('%')
    features['nb_slash'] = url.count('/')
    features['nb_star'] = url.count('*')
    features['nb_colon'] = url.count(':')
    features['nb_comma'] = url.count(',')
    features['nb_semicolumn'] = url.count(';')
    features['nb_dollar'] = url.count('$')
    features['nb_space'] = url.count(' ')
    features['nb_www'] = url.count('www')
    features['nb_com'] = url.count('.com')
    features['nb_dslash'] = url.count('//')

    # Path Features
    features['http_in_path'] = 'http' in parsed_url.path
    features['https_token'] = 'https' in url
    features['ratio_digits_url'] = sum(c.isdigit() for c in url) / len(url) if len(url) > 0 else 0

    # Domain Features
    features['punycode'] = parsed_url.hostname.encode('idna').decode('utf-8') != parsed_url.hostname
    features['shortening_service'] = any(service in url for service in ['bit.ly', 'tinyurl.com', 'goo.gl'])
    features['port'] = parsed_url.port if parsed_url.port else -1
    features['tld_in_path'] = any(tld in parsed_url.path for tld in ['.com', '.net', '.org'])
    features['tld_in_subdomain'] = domain_info.suffix in parsed_url.hostname
    features['path_extension'] = any(ext in parsed_url.path for ext in ['.php', '.asp', '.aspx', '.jsp'])
    features['prefix_suffix'] = any(part in url for part in ['.htm', '.html'])
    features['random_domain'] = any(sub in parsed_url.hostname for sub in ['random', '1234', 'xyz'])

    # Fetch Page Content for More Features
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Count Links in the Page
        links = soup.find_all('a')
        features['nb_hyperlinks'] = len(links)
        features['ratio_extHyperlinks'] = len([link for link in links if link.get('href', '').startswith('http')]) / len(links) if len(links) > 0 else 0
        features['ratio_intHyperlinks'] = len([link for link in links if link.get('href', '') and not link.get('href', '').startswith('http')]) / len(links) if len(links) > 0 else 0
        features['ratio_nullHyperlinks'] = len([link for link in links if not link.get('href')]) / len(links) if len(links) > 0 else 0

        # Check for Login Form
        features['login_form'] = int(bool(soup.find('form') and (soup.find('input', {'type': 'password'}) or soup.find('input', {'type': 'text'}))))

        # Check for External CSS
        css_links = soup.find_all('link', {'rel': 'stylesheet'})
        features['nb_extCSS'] = len(css_links)

        # External Links and Media
        media_links = soup.find_all(['img', 'video', 'audio'])
        features['ratio_extMedia'] = len(media_links) / len(links) if len(links) > 0 else 0

        # Check for Popups and Iframes
        features['iframe'] = int(bool(soup.find('iframe')))
        features['popup_window'] = int(bool(soup.find('script', text=re.compile('window.open'))))

        # Additional Features
        features['sfh'] = int(bool(soup.find('form', action=lambda x: x and x.startswith('http'))))
        features['safe_anchor'] = int(all(link.get('rel') != 'nofollow' for link in links))
        features['onmouseover'] = int(bool(soup.find_all(attrs={"onmouseover": True})))
        features['right_clic'] = int(bool(soup.find_all(attrs={"oncontextmenu": True})))
        features['empty_title'] = int(bool(soup.find('title') and not soup.find('title').text.strip()))
        features['domain_in_title'] = int(domain_info.domain in (soup.find('title').text if soup.find('title') else ''))
        features['domain_with_copyright'] = int('copyright' in (soup.find('body').text if soup.find('body') else '').lower())

    except Exception as e:
        print(f"Error fetching URL content: {e}")
        features['nb_hyperlinks'] = 0
        features['ratio_extHyperlinks'] = 0
        features['ratio_intHyperlinks'] = 0
        features['ratio_nullHyperlinks'] = 0
        features['nb_extCSS'] = 0
        features['ratio_extMedia'] = 0
        features['iframe'] = 0
        features['popup_window'] = 0
        features['sfh'] = 0
        features['safe_anchor'] = 0
        features['onmouseover'] = 0
        features['right_clic'] = 0
        features['empty_title'] = 0
        features['domain_in_title'] = 0
        features['domain_with_copyright'] = 0

    # WHOIS Registration Features
    try:
        whois_info = whois.whois(parsed_url.hostname)
        features['whois_registered_domain'] = int(bool(whois_info.domain_name))
        features['domain_registration_length'] = whois_info.domain_age if whois_info.domain_age else -1
        features['domain_age'] = (datetime.datetime.now() - whois_info.creation_date).days if whois_info.creation_date else -1
    except Exception as e:
        print(f"Error fetching WHOIS info: {e}")
        features['whois_registered_domain'] = 0
        features['domain_registration_length'] = -1
        features['domain_age'] = -1

    return features

# Example usage
url = 'https://www.example.com'
extracted_features = extract_features(url)

# Print features in list format
feature_list = [f'{key}: {value}' for key, value in extracted_features.items()]
for feature in feature_list:
    print(feature)


In [ ]:
import numpy as np
import pandas as pd
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import re
import whois
import tldextract
import datetime
import socket
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def is_valid_ip(ip):
    """Check if the hostname is an IP address"""
    try:
        socket.inet_aton(ip)
        return True
    except socket.error:
        return False

def extract_features(url):
    features = {}

    # Basic URL parsing
    parsed_url = urlparse(url)
    domain_info = tldextract.extract(url)

    # Basic URL Length Features
    features['length_url'] = len(url)
    features['length_hostname'] = len(parsed_url.hostname) if parsed_url.hostname else 0

    # Token Counts
    tokens = ['.', '-', '@', '?', '&', '|', '=', '_', '~', '%', '/', '*', ':', ',', ';', '$', ' ']
    token_names = ['nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore',
                   'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma',
                   'nb_semicolumn', 'nb_dollar', 'nb_space']

    for token, name in zip(tokens, token_names):
        features[name] = url.count(token)

    features['nb_www'] = url.count('www')
    features['nb_com'] = url.count('.com')
    features['nb_dslash'] = url.count('//')

    # Path Features
    features['http_in_path'] = 'http' in parsed_url.path
    features['https_token'] = 'https' in url
    features['ratio_digits_url'] = sum(c.isdigit() for c in url) / len(url) if len(url) > 0 else 0

    path_extensions = ['.php', '.asp', '.aspx', '.jsp']
    features['path_extension'] = any(ext in parsed_url.path for ext in path_extensions)
    features['prefix_suffix'] = any(part in url for part in ['.htm', '.html'])
    random_strings = ['random', '1234', 'xyz']
    features['random_domain'] = any(sub in parsed_url.hostname for sub in random_strings)

    # Domain Features
    features['punycode'] = parsed_url.hostname.encode('idna').decode('utf-8') != parsed_url.hostname
    shortening_services = ['bit.ly', 'tinyurl.com', 'goo.gl']
    features['shortening_service'] = any(service in url for service in shortening_services)
    features['port'] = parsed_url.port if parsed_url.port else -1
    tlds = ['.com', '.net', '.org']
    features['tld_in_path'] = any(tld in parsed_url.path for tld in tlds)
    features['tld_in_subdomain'] = domain_info.suffix in parsed_url.hostname

    # Check if IP address is used as hostname
    features['ip'] = int(is_valid_ip(parsed_url.hostname))

    # Ratio of digits in hostname
    features['ratio_digits_host'] = sum(c.isdigit() for c in parsed_url.hostname) / len(parsed_url.hostname) if len(parsed_url.hostname) > 0 else 0

    # Abnormal subdomain
    features['abnormal_subdomain'] = int(len(domain_info.subdomain.split('.')) > 2)

    # Number of subdomains
    features['nb_subdomains'] = len(domain_info.subdomain.split('.')) if domain_info.subdomain else 0

    # Initialize media ratios
    features['ratio_intMedia'] = 0
    features['ratio_extMedia'] = 0

    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Count Links in the Page
        links = soup.find_all('a')
        total_links = len(links)
        features['nb_hyperlinks'] = total_links

        if total_links > 0:
            ext_links = len([link for link in links if link.get('href', '').startswith('http')])
            int_links = len([link for link in links if not link.get('href', '').startswith('http')])
            null_links = len([link for link in links if not link.get('href')])

            features['ratio_extHyperlinks'] = ext_links / total_links
            features['ratio_intHyperlinks'] = int_links / total_links
            features['ratio_nullHyperlinks'] = null_links / total_links
        else:
            features['ratio_extHyperlinks'] = 0
            features['ratio_intHyperlinks'] = 0
            features['ratio_nullHyperlinks'] = 0

        # Calculate redirections
        features['nb_redirection'] = len(response.history)
        features['nb_external_redirection'] = len([resp for resp in response.history if urlparse(resp.url).netloc != parsed_url.netloc])

        # Internal and external redirection ratios
        features['ratio_intRedirection'] = len([resp for resp in response.history if urlparse(resp.url).netloc == parsed_url.netloc]) / len(response.history) if response.history else 0
        features['ratio_extRedirection'] = features['nb_external_redirection'] / len(response.history) if response.history else 0

        # Check for favicon
        favicon_tag = soup.find('link', rel='shortcut icon')
        features['external_favicon'] = int(favicon_tag and parsed_url.netloc not in favicon_tag.get('href', ''))

        # Links in specific tags
        script_links = len(soup.find_all('script', src=True))
        link_links = len(soup.find_all('link', href=True))
        features['links_in_tags'] = script_links + link_links

        features['login_form'] = int(bool(soup.find('form') and
                                            (soup.find('input', {'type': 'password'}) or
                                             soup.find('input', {'type': 'text'}))))

        # External CSS
        css_links = soup.find_all('link', {'rel': 'stylesheet'})
        features['nb_extCSS'] = len(css_links)

        media_links = soup.find_all(['img', 'video', 'audio'])
        features['ratio_extMedia'] = len(media_links) / len(links) if len(links) > 0 else 0
        features['iframe'] = int(bool(soup.find('iframe')))
        features['popup_window'] = int(bool(soup.find('script', text=re.compile('window.open'))))

        safe_anchors = [link.get('rel') != 'nofollow' for link in links]
        features['sfh'] = int(any(form.get('action', '').startswith('http') for form in soup.find_all('form')))
        features['safe_anchor'] = int(all(safe_anchors))

        features['onmouseover'] = int(bool(soup.find_all(attrs={"onmouseover": True})))
        features['right_clic'] = int(bool(soup.find_all(attrs={"oncontextmenu": True})))
        features['empty_title'] = int(bool(soup.find('title') and not soup.find('title').text.strip()))
        features['domain_in_title'] = int(domain_info.domain in (soup.find('title').text if soup.find('title') else ''))
        features['domain_with_copyright'] = int('copyright' in (soup.find('body').text if soup.find('body') else '').lower())

    except Exception as e:
        print(f"Error fetching URL content: {e}")
        # Set defaults for features related to URL fetching
        keys = ['nb_hyperlinks', 'ratio_extHyperlinks', 'ratio_intHyperlinks', 'ratio_nullHyperlinks',
                'nb_redirection', 'nb_external_redirection', 'ratio_intRedirection', 'ratio_extRedirection',
                'links_in_tags', 'external_favicon', 'iframe', 'popup_window', 'sfh', 'safe_anchor',
                'onmouseover', 'right_clic', 'empty_title', 'domain_in_title', 'domain_with_copyright']
        for key in keys:
            features[key] = 0

    # Calculate the lengths of words
    path_words = re.findall(r'\w+', parsed_url.path)
    host_words = re.findall(r'\w+', parsed_url.hostname)
    all_words = path_words + host_words
    features['length_words_raw'] = len(' '.join(all_words))

    features['char_repeat'] = len({char: url.count(char) for char in set(url) if url.count(char) > 1})

    # Capture lengths of words
    features['shortest_word_host'] = len(min(host_words, key=len, default=''))
    features['shortest_word_path'] = len(min(path_words, key=len, default=''))
    features['longest_word_host'] = len(max(host_words, key=len, default=''))
    features['longest_word_path'] = len(max(path_words, key=len, default=''))

    features['shortest_words_raw'] = len(min(all_words, key=len, default=''))
    features['longest_words_raw'] = len(max(all_words, key=len, default=''))
    features['avg_word_host'] = np.mean([len(word) for word in host_words]) if host_words else 0
    features['avg_word_path'] = np.mean([len(word) for word in path_words]) if path_words else 0
    features['avg_words_raw'] = np.mean([len(word) for word in all_words]) if all_words else 0

    features['phish_hints'] = int(any(hint in url for hint in ['secure', 'login', 'update', 'verify']))

    features['domain_in_brand'] = int(domain_info.domain in url)
    features['brand_in_subdomain'] = int(any(brand in parsed_url.hostname for brand in ['brand', 'company']))
    features['brand_in_path'] = int(any(brand in parsed_url.path for brand in ['brand', 'company']))

    suspicious_tlds = ['.tk', '.ml', '.ga', '.cf', '.gq']
    features['suspecious_tld'] = int(any(tld in parsed_url.hostname for tld in suspicious_tlds))

    # Placeholder for Int Errors and Ext Errors
    features['ratio_intErrors'] = 0  # Placeholder, logic needed to calculate
    features['ratio_extErrors'] = 0  # Placeholder, logic needed to calculate
    features['submit_email'] = int(bool(soup.find('input', {'type': 'email'})))

    try:
        whois_info = whois.whois(parsed_url.hostname)
        features['whois_registered_domain'] = int(bool(whois_info.domain_name))
        if whois_info.creation_date and whois_info.expiration_date:
            if isinstance(whois_info.creation_date, list):
                creation_date = whois_info.creation_date[0]
            else:
                creation_date = whois_info.creation_date

            if isinstance(whois_info.expiration_date, list):
                expiration_date = whois_info.expiration_date[0]
            else:
                expiration_date = whois_info.expiration_date

            features['domain_registration_length'] = (expiration_date - creation_date).days
            features['domain_age'] = (datetime.datetime.now() - creation_date).days
        else:
            features['domain_registration_length'] = -1
            features['domain_age'] = -1
    except Exception as e:
        print(f"Error fetching WHOIS info: {e}")
        features['whois_registered_domain'] = 0
        features['domain_registration_length'] = -1
        features['domain_age'] = -1

    return features

# Load and preprocess data
data_path = '/content/drive/MyDrive/dataset_phishing.csv'  # Adjust this to your data path
df = pd.read_csv(data_path, encoding='utf-8')
df.dropna(inplace=True)

# Map the 'status' column to numeric values before splitting the data
df['status'] = df['status'].map({'legitimate': 0, 'phishing': 1})

# Feature and target selection
feature_columns = [ 'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq',
    'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma', 'nb_semicolumn',
    'nb_dollar', 'nb_space', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url',
    'ratio_digits_host', 'punycode', 'port', 'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains',
    'prefix_suffix', 'random_domain', 'shortening_service', 'path_extension', 'nb_redirection', 'nb_external_redirection',
    'length_words_raw', 'char_repeat', 'shortest_words_raw', 'shortest_word_host', 'shortest_word_path',
    'longest_words_raw', 'longest_word_host', 'longest_word_path', 'avg_words_raw', 'avg_word_host', 'avg_word_path',
    'phish_hints', 'domain_in_brand', 'brand_in_subdomain', 'brand_in_path', 'suspecious_tld',
    'nb_hyperlinks', 'ratio_intHyperlinks', 'ratio_extHyperlinks', 'ratio_nullHyperlinks', 'nb_extCSS',
    'ratio_intRedirection', 'ratio_extRedirection', 'ratio_intErrors', 'ratio_extErrors', 'login_form',
    'external_favicon', 'links_in_tags', 'submit_email', 'ratio_intMedia', 'ratio_extMedia', 'sfh', 'iframe',
    'popup_window', 'safe_anchor', 'onmouseover', 'right_clic', 'empty_title', 'domain_in_title',
    'domain_with_copyright', 'whois_registered_domain', 'domain_registration_length', 'domain_age']

# Selecting features
X = df[feature_columns].values
y = df['status'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 128
num_layers = 2
output_size = 1
num_epochs = 100
learning_rate = 0.005

# Initialize the model, loss function, and optimizer
model = RNNModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train.unsqueeze(1))  # Reshape for RNN
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    y_pred_train = model(X_train.unsqueeze(1)).round()
    y_pred_test = model(X_test.unsqueeze(1)).round()

    train_accuracy = accuracy_score(y_train, y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    print(f'Training Accuracy: {train_accuracy:.4f}')
    print(f'Test Accuracy: {test_accuracy:.4f}')

    print('Classification Report:')
    print(classification_report(y_test, y_pred_test))

    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred_test))

# Predicting new URL
new_url = 'https://www.shadetreetechnology.com/V4/validation/ba4b8bddd7958ecb8772c836c2969531'  # Change this to any URL you want to test
new_features = extract_features(new_url)

# Extract feature values into a list
feature_values_list = list(new_features.values())

# If you want to convert the list to a DataFrame for scaling
new_features_df = pd.DataFrame([feature_values_list])
new_features_scaled = scaler.transform(new_features_df)

# Convert to PyTorch tensor
new_features_tensor = torch.tensor(new_features_scaled, dtype=torch.float32).unsqueeze(1)

# Make prediction
model.eval()
with torch.no_grad():
    new_prediction = model(new_features_tensor).round()
    print(f'Prediction for {new_url}: {"Phishing" if new_prediction.item() == 1 else "Legitimate"}')

# Print extracted feature values list
print(f'Extracted feature values: {feature_values_list}')

Epoch [10/100], Loss: 0.2766
Epoch [20/100], Loss: 0.2393
Epoch [30/100], Loss: 0.2263
Epoch [40/100], Loss: 0.2139
Epoch [50/100], Loss: 0.1987
Epoch [60/100], Loss: 0.1775
Epoch [70/100], Loss: 0.1535
Epoch [80/100], Loss: 0.1298
Epoch [90/100], Loss: 0.1120
Epoch [100/100], Loss: 0.0933
Training Accuracy: 0.9682
Test Accuracy: 0.9418
Classification Report:
              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94      1157
         1.0       0.94      0.94      0.94      1129

    accuracy                           0.94      2286
   macro avg       0.94      0.94      0.94      2286
weighted avg       0.94      0.94      0.94      2286

Confusion Matrix:
[[1091   66]
 [  67 1062]]
Error fetching URL content: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'


ValueError: X has 80 features, but StandardScaler is expecting 82 features as input.

In [ ]:
pip install numpy pandas requests beautifulsoup4 python-whois tldextract torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 5.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import re
import whois
import tldextract
import datetime
import socket
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib
def is_valid_ip(ip):
    """Check if the hostname is an IP address."""
    try:
        socket.inet_aton(ip)
        return True
    except socket.error:
        return False

def extract_features(url):
    features = {}

    # Basic URL parsing
    parsed_url = urlparse(url)
    domain_info = tldextract.extract(url)

    # Basic URL Length Features
    features['length_url'] = len(url)
    features['length_hostname'] = len(parsed_url.hostname or '')

    # Token Counts
    tokens = ['.', '-', '@', '?', '&', '|', '=', '_', '~', '%', '/', '*', ':', ',', ';', '$', ' ']
    token_names = ['nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore',
                   'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma',
                   'nb_semicolumn', 'nb_dollar', 'nb_space']

    for token, name in zip(tokens, token_names):
        features[name] = url.count(token)

    features['nb_www'] = url.count('www')
    features['nb_com'] = url.count('.com')
    features['nb_dslash'] = url.count('//')

    # Path Features
    features['http_in_path'] = int('http' in parsed_url.path)
    features['https_token'] = int('https' in url)
    features['ratio_digits_url'] = sum(c.isdigit() for c in url) / len(url) if len(url) > 0 else 0

    path_extensions = ['.php', '.asp', '.aspx', '.jsp']
    features['path_extension'] = int(any(ext in parsed_url.path for ext in path_extensions))
    features['prefix_suffix'] = int(any(part in url for part in ['.htm', '.html']))
    random_strings = ['random', '1234', 'xyz']
    features['random_domain'] = int(any(sub in parsed_url.hostname for sub in random_strings)) if parsed_url.hostname else 0

    # Domain Features
    features['punycode'] = int(parsed_url.hostname.encode('idna').decode('utf-8') != parsed_url.hostname) if parsed_url.hostname else 0
    shortening_services = ['bit.ly', 'tinyurl.com', 'goo.gl']
    features['shortening_service'] = int(any(service in url for service in shortening_services))
    features['port'] = parsed_url.port if parsed_url.port else -1
    tlds = ['.com', '.net', '.org']
    features['tld_in_path'] = int(any(tld in parsed_url.path for tld in tlds))
    features['tld_in_subdomain'] = int(domain_info.suffix in parsed_url.hostname) if parsed_url.hostname else 0

    # Check if IP address is used as hostname
    features['ip'] = int(is_valid_ip(parsed_url.hostname)) if parsed_url.hostname else 0

    # Ratio of digits in hostname
    features['ratio_digits_host'] = sum(c.isdigit() for c in parsed_url.hostname) / len(parsed_url.hostname) if parsed_url.hostname and len(parsed_url.hostname) > 0 else 0

    # Abnormal subdomain
    features['abnormal_subdomain'] = int(len(domain_info.subdomain.split('.')) > 2) if domain_info.subdomain else 0

    # Number of subdomains
    features['nb_subdomains'] = len(domain_info.subdomain.split('.')) if domain_info.subdomain else 0

    # Initialize media ratios
    features['ratio_intMedia'] = 0
    features['ratio_extMedia'] = 0

    # Default values for URL content-based features
    default_features = {
        'nb_hyperlinks': 0,
        'ratio_extHyperlinks': 0,
        'ratio_intHyperlinks': 0,
        'ratio_nullHyperlinks': 0,
        'nb_redirection': 0,
        'nb_external_redirection': 0,
        'ratio_intRedirection': 0,
        'ratio_extRedirection': 0,
        'links_in_tags': 0,
        'external_favicon': 0,
        'iframe': 0,
        'popup_window': 0,
        'sfh': 0,
        'safe_anchor': 0,
        'onmouseover': 0,
        'right_clic': 0,
        'empty_title': 0,
        'domain_in_title': 0,
        'domain_with_copyright': 0
    }

    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Count Links in the Page
        links = soup.find_all('a')
        total_links = len(links)
        features['nb_hyperlinks'] = total_links

        if total_links > 0:
            ext_links = len([link for link in links if link.get('href', '').startswith('http')])
            int_links = len([link for link in links if not link.get('href', '').startswith('http')])
            null_links = len([link for link in links if not link.get('href')])

            features['ratio_extHyperlinks'] = ext_links / total_links
            features['ratio_intHyperlinks'] = int_links / total_links
            features['ratio_nullHyperlinks'] = null_links / total_links
        else:
            features['ratio_extHyperlinks'] = 0
            features['ratio_intHyperlinks'] = 0
            features['ratio_nullHyperlinks'] = 0

        # Calculate redirections
        features['nb_redirection'] = len(response.history)
        features['nb_external_redirection'] = len([resp for resp in response.history if urlparse(resp.url).netloc != parsed_url.netloc])

        # Internal and external redirection ratios
        features['ratio_intRedirection'] = len([resp for resp in response.history if urlparse(resp.url).netloc == parsed_url.netloc]) / len(response.history) if response.history else 0
        features['ratio_extRedirection'] = features['nb_external_redirection'] / len(response.history) if response.history else 0

        # Check for favicon
        favicon_tag = soup.find('link', rel='shortcut icon')
        features['external_favicon'] = int(favicon_tag and parsed_url.netloc not in favicon_tag.get('href', ''))

        # Links in specific tags
        script_links = len(soup.find_all('script', src=True))
        link_links = len(soup.find_all('link', href=True))
        features['links_in_tags'] = script_links + link_links

        media_links = soup.find_all(['img', 'video', 'audio'])
        features['ratio_extMedia'] = len(media_links) / len(links) if len(links) > 0 else 0
        features['iframe'] = int(bool(soup.find('iframe')))
        features['popup_window'] = int(bool(soup.find('script', string=re.compile('window.open'))))

        safe_anchors = [link.get('rel') != 'nofollow' for link in links]
        features['sfh'] = int(any(form.get('action', '').startswith('http') for form in soup.find_all('form')))
        features['safe_anchor'] = int(all(safe_anchors))

        features['onmouseover'] = int(bool(soup.find_all(attrs={"onmouseover": True})))
        features['right_clic'] = int(bool(soup.find_all(attrs={"oncontextmenu": True})))
        features['empty_title'] = int(bool(soup.find('title') and not soup.find('title').text.strip()))
        features['domain_in_title'] = int(domain_info.domain in (soup.find('title').text if soup.find('title') else ''))
        features['domain_with_copyright'] = int('copyright' in (soup.find('body').text if soup.find('body') else '').lower())

    except Exception as e:
        print(f"Error fetching URL content: {e}")
        features.update(default_features)

    # Calculate the lengths of words
    path_words = re.findall(r'\w+', parsed_url.path)
    host_words = re.findall(r'\w+', parsed_url.hostname) if parsed_url.hostname else []
    all_words = path_words + host_words
    features['length_words_raw'] = len(' '.join(all_words))

    features['char_repeat'] = len({char: url.count(char) for char in set(url) if url.count(char) > 1})

    # Capture lengths of words
    features['shortest_word_host'] = len(min(host_words, key=len, default=''))
    features['shortest_word_path'] = len(min(path_words, key=len, default=''))
    features['longest_word_host'] = len(max(host_words, key=len, default=''))
    features['longest_word_path'] = len(max(path_words, key=len, default=''))

    features['shortest_words_raw'] = len(min(all_words, key=len, default=''))
    features['longest_words_raw'] = len(max(all_words, key=len, default=''))
    features['avg_word_host'] = np.mean([len(word) for word in host_words]) if host_words else 0
    features['avg_word_path'] = np.mean([len(word) for word in path_words]) if path_words else 0
    features['avg_words_raw'] = np.mean([len(word) for word in all_words]) if all_words else 0

    features['phish_hints'] = int(any(hint in url for hint in ['secure', 'login', 'update', 'verify']))

    features['domain_in_brand'] = int(domain_info.domain in url)
    features['brand_in_subdomain'] = int(any(brand in parsed_url.hostname for brand in ['brand', 'company'])) if parsed_url.hostname else 0
    features['brand_in_path'] = int(any(brand in parsed_url.path for brand in ['brand', 'company']))

    suspicious_tlds = ['.tk', '.ml', '.ga', '.cf', '.gq']
    features['suspecious_tld'] = int(any(tld in parsed_url.hostname for tld in suspicious_tlds)) if parsed_url.hostname else 0

    # Placeholder for Int Errors and Ext Errors
    features['ratio_intErrors'] = 0  # Placeholder, logic needed to calculate
    features['ratio_extErrors'] = 0  # Placeholder, logic needed to calculate
    features['submit_email'] = int(bool(soup.find('input', {'type': 'email'}))) if 'soup' in locals() else 0

    try:
        whois_info = whois.whois(parsed_url.hostname)
        features['whois_registered_domain'] = int(bool(whois_info.domain_name))
        if whois_info.creation_date and whois_info.expiration_date:
            if isinstance(whois_info.creation_date, list):
                creation_date = whois_info.creation_date[0]
            else:
                creation_date = whois_info.creation_date

            if isinstance(whois_info.expiration_date, list):
                expiration_date = whois_info.expiration_date[0]
            else:
                expiration_date = whois_info.expiration_date

            features['domain_registration_length'] = (expiration_date - creation_date).days
            features['domain_age'] = (datetime.datetime.now() - creation_date).days
        else:
            features['domain_registration_length'] = -1
            features['domain_age'] = -1
    except Exception as e:
        print(f"Error fetching WHOIS info: {e}")
        features['whois_registered_domain'] = 0
        features['domain_registration_length'] = -1
        features['domain_age'] = -1

    return features

# Load and preprocess data
data_path = '/content/drive/MyDrive/dataset_phishing.csv'  # Adjust this to your data path
df = pd.read_csv(data_path, encoding='utf-8')
df.dropna(inplace=True)


# Map the 'status' column to numeric values before splitting the data
df['status'] = df['status'].map({'legitimate': 0, 'phishing': 1})

# Feature and target selection
feature_columns = [
     'url', 'length_url', 'length_hostname',  # Added 'url' as a feature temporarily
    'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq',
    'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon',
    'nb_comma', 'nb_semicolumn', 'nb_dollar', 'nb_space', 'nb_www', 'nb_com',
    'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url', 'ratio_digits_host',
    'punycode', 'port', 'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain',
    'nb_subdomains', 'prefix_suffix', 'random_domain', 'shortening_service',
    'path_extension', 'nb_redirection', 'nb_external_redirection', 'length_words_raw',
    'char_repeat', 'shortest_words_raw', 'shortest_word_host', 'shortest_word_path',
    'longest_words_raw', 'longest_word_host', 'longest_word_path', 'avg_words_raw',
    'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand',
    'brand_in_subdomain', 'brand_in_path', 'suspecious_tld', 'nb_hyperlinks',
    'ratio_intHyperlinks', 'ratio_extHyperlinks', 'ratio_nullHyperlinks',
    'ratio_intRedirection', 'ratio_extRedirection', 'ratio_intErrors', 'ratio_extErrors',
  'external_favicon', 'links_in_tags', 'submit_email',
    'ratio_intMedia', 'ratio_extMedia', 'sfh', 'iframe', 'popup_window',
    'safe_anchor', 'onmouseover', 'right_clic', 'empty_title', 'domain_in_title',
    'domain_with_copyright', 'whois_registered_domain', 'domain_registration_length',
    'domain_age'
]

# Selecting features without including 'url' for model training
X = df[feature_columns].drop(columns=['url']).values  # Exclude the 'url' column
y = df['status'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 128
num_layers = 2
output_size = 1
num_epochs = 100
learning_rate = 0.005

# Initialize the model, loss function, and optimizer
model = RNNModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train.unsqueeze(1))  # Reshape for RNN
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    y_pred_train = model(X_train.unsqueeze(1)).round()
    y_pred_test = model(X_test.unsqueeze(1)).round()

    train_accuracy = accuracy_score(y_train, y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    print(f'Training Accuracy: {train_accuracy:.4f}')
    print(f'Test Accuracy: {test_accuracy:.4f}')

    print('Classification Report:')
    print(classification_report(y_test, y_pred_test))

    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred_test))

# Predicting new URL
urls = [
    'https://now-get-free-verified.vercel.app/',
    'https://s.monsoondealz.xyz/',
    'http://dpd-hr.receiving-delivery.com/track/5387861827/',
    'https://www.google.com/,',
    'https://www.facebook.com/',
    'http://rgipt.ac.in',
    'https://www.youtube.com/',


]

# Extract features for each URL
feature_values_list = []
for url in urls:
    new_features = extract_features(url)

    # Extract feature values into a list
    feature_values = [new_features[col] for col in feature_columns if col != 'url']

    # Append to the list
    feature_values_list.append(feature_values)

# Create DataFrame for the new features
new_features_df = pd.DataFrame(feature_values_list, columns=[col for col in feature_columns if col != 'url'])

# Scale the features
new_features_scaled = scaler.transform(new_features_df)

# Convert to PyTorch tensor
new_features_tensor = torch.tensor(new_features_scaled, dtype=torch.float32).unsqueeze(1)

# Make predictions
model.eval()
with torch.no_grad():
    predictions = model(new_features_tensor).round()

# Print predictions for each URL
for url, pred in zip(urls, predictions):
    print(f'Prediction for {url}: {"Phishing" if pred.item() == 1 else "Legitimate"}')

# Print extracted feature values list for debugging
for url, features in zip(urls, feature_values_list):
    print(f'Extracted feature values for {url}: {features}')


model_path = '/content/drive/MyDrive/sweeb11.pth'
scaler_path = '/content/drive/MyDrive/sweb12.pkl'

# Save the model
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

# Save the scaler
joblib.dump(scaler, scaler_path)
print(f"Scaler saved to {scaler_path}")

# Confirm the files exist
import os
print("Files in drive:")
print(os.listdir('/content/drive/MyDrive'))


Epoch [10/100], Loss: 0.2767
Epoch [20/100], Loss: 0.2407
Epoch [30/100], Loss: 0.2288
Epoch [40/100], Loss: 0.2198
Epoch [50/100], Loss: 0.2087
Epoch [60/100], Loss: 0.1921
Epoch [70/100], Loss: 0.1704
Epoch [80/100], Loss: 0.1467
Epoch [90/100], Loss: 0.1243
Epoch [100/100], Loss: 0.1036
Training Accuracy: 0.9612
Test Accuracy: 0.9396
Classification Report:
              precision    recall  f1-score   support

         0.0       0.95      0.93      0.94      1157
         1.0       0.93      0.95      0.94      1129

    accuracy                           0.94      2286
   macro avg       0.94      0.94      0.94      2286
weighted avg       0.94      0.94      0.94      2286

Confusion Matrix:
[[1079   78]
 [  60 1069]]
Error fetching URL content: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'
Error fetching URL content: HTTPSConnectionPool(host='s.monsoondealz.xyz', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [ ]:
import numpy as np
import pandas as pd
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import re
import whois
import tldextract
import datetime
import socket
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

def is_valid_ip(ip):
    """Check if the hostname is an IP address."""
    try:
        socket.inet_aton(ip)
        return True
    except socket.error:
        return False

def extract_features(url):
    features = {}

    # Basic URL parsing
    parsed_url = urlparse(url)
    domain_info = tldextract.extract(url)

    # Basic URL Length Features
    features['length_url'] = len(url)
    features['length_hostname'] = len(parsed_url.hostname or '')

    # Token Counts
    tokens = ['.', '-', '@', '?', '&', '|', '=', '_', '~', '%', '/', '*', ':', ',', ';', '$', ' ']
    token_names = ['nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore',
                   'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma',
                   'nb_semicolumn', 'nb_dollar', 'nb_space']

    for token, name in zip(tokens, token_names):
        features[name] = url.count(token)

    features['nb_www'] = url.count('www')
    features['nb_com'] = url.count('.com')
    features['nb_dslash'] = url.count('//')

    # Path Features
    features['http_in_path'] = int('http' in parsed_url.path)
    features['https_token'] = int('https' in url)
    features['ratio_digits_url'] = sum(c.isdigit() for c in url) / len(url) if len(url) > 0 else 0

    path_extensions = ['.php', '.asp', '.aspx', '.jsp']
    features['path_extension'] = int(any(ext in parsed_url.path for ext in path_extensions))
    features['prefix_suffix'] = int(any(part in url for part in ['.htm', '.html']))
    random_strings = ['random', '1234', 'xyz']
    features['random_domain'] = int(any(sub in parsed_url.hostname for sub in random_strings)) if parsed_url.hostname else 0

    # Domain Features
    features['punycode'] = int(parsed_url.hostname.encode('idna').decode('utf-8') != parsed_url.hostname) if parsed_url.hostname else 0
    shortening_services = ['bit.ly', 'tinyurl.com', 'goo.gl']
    features['shortening_service'] = int(any(service in url for service in shortening_services))
    features['port'] = parsed_url.port if parsed_url.port else -1
    tlds = ['.com', '.net', '.org']
    features['tld_in_path'] = int(any(tld in parsed_url.path for tld in tlds))
    features['tld_in_subdomain'] = int(domain_info.suffix in parsed_url.hostname) if parsed_url.hostname else 0

    # Check if IP address is used as hostname
    features['ip'] = int(is_valid_ip(parsed_url.hostname)) if parsed_url.hostname else 0

    # Ratio of digits in hostname
    features['ratio_digits_host'] = sum(c.isdigit() for c in parsed_url.hostname) / len(parsed_url.hostname) if parsed_url.hostname and len(parsed_url.hostname) > 0 else 0

    # Abnormal subdomain
    features['abnormal_subdomain'] = int(len(domain_info.subdomain.split('.')) > 2) if domain_info.subdomain else 0

    # Number of subdomains
    features['nb_subdomains'] = len(domain_info.subdomain.split('.')) if domain_info.subdomain else 0

    # Initialize media ratios
    features['ratio_intMedia'] = 0
    features['ratio_extMedia'] = 0

    # Default values for URL content-based features
    default_features = {
        'nb_hyperlinks': 0,
        'ratio_extHyperlinks': 0,
        'ratio_intHyperlinks': 0,
        'ratio_nullHyperlinks': 0,
        'nb_redirection': 0,
        'nb_external_redirection': 0,
        'ratio_intRedirection': 0,
        'ratio_extRedirection': 0,
        'links_in_tags': 0,
        'external_favicon': 0,
        'iframe': 0,
        'popup_window': 0,
        'sfh': 0,
        'safe_anchor': 0,
        'onmouseover': 0,
        'right_clic': 0,
        'empty_title': 0,
        'domain_in_title': 0,
        'domain_with_copyright': 0
    }

    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Count Links in the Page
        links = soup.find_all('a')
        total_links = len(links)
        features['nb_hyperlinks'] = total_links

        if total_links > 0:
            ext_links = len([link for link in links if link.get('href', '').startswith('http')])
            int_links = len([link for link in links if not link.get('href', '').startswith('http')])
            null_links = len([link for link in links if not link.get('href')])

            features['ratio_extHyperlinks'] = ext_links / total_links
            features['ratio_intHyperlinks'] = int_links / total_links
            features['ratio_nullHyperlinks'] = null_links / total_links
        else:
            features['ratio_extHyperlinks'] = 0
            features['ratio_intHyperlinks'] = 0
            features['ratio_nullHyperlinks'] = 0

        # Calculate redirections
        features['nb_redirection'] = len(response.history)
        features['nb_external_redirection'] = len([resp for resp in response.history if urlparse(resp.url).netloc != parsed_url.netloc])

        # Internal and external redirection ratios
        features['ratio_intRedirection'] = len([resp for resp in response.history if urlparse(resp.url).netloc == parsed_url.netloc]) / len(response.history) if response.history else 0
        features['ratio_extRedirection'] = features['nb_external_redirection'] / len(response.history) if response.history else 0

        # Check for favicon
        favicon_tag = soup.find('link', rel='shortcut icon')
        features['external_favicon'] = int(favicon_tag and parsed_url.netloc not in favicon_tag.get('href', ''))

        # Links in specific tags
        script_links = len(soup.find_all('script', src=True))
        link_links = len(soup.find_all('link', href=True))
        features['links_in_tags'] = script_links + link_links

        media_links = soup.find_all(['img', 'video', 'audio'])
        features['ratio_extMedia'] = len(media_links) / len(links) if len(links) > 0 else 0
        features['iframe'] = int(bool(soup.find('iframe')))
        features['popup_window'] = int(bool(soup.find('script', string=re.compile('window.open'))))

        safe_anchors = [link.get('rel') != 'nofollow' for link in links]
        features['sfh'] = int(any(form.get('action', '').startswith('http') for form in soup.find_all('form')))
        features['safe_anchor'] = int(all(safe_anchors))

        features['onmouseover'] = int(bool(soup.find_all(attrs={"onmouseover": True})))
        features['right_clic'] = int(bool(soup.find_all(attrs={"oncontextmenu": True})))
        features['empty_title'] = int(bool(soup.find('title') and not soup.find('title').text.strip()))
        features['domain_in_title'] = int(domain_info.domain in (soup.find('title').text if soup.find('title') else ''))
        features['domain_with_copyright'] = int('copyright' in (soup.find('body').text if soup.find('body') else '').lower())

    except Exception as e:
        print(f"Error fetching URL content: {e}")
        features.update(default_features)

    # Calculate the lengths of words
    path_words = re.findall(r'\w+', parsed_url.path)
    host_words = re.findall(r'\w+', parsed_url.hostname) if parsed_url.hostname else []
    all_words = path_words + host_words
    features['length_words_raw'] = len(' '.join(all_words))

    features['char_repeat'] = len({char: url.count(char) for char in set(url) if url.count(char) > 1})

    # Capture lengths of words
    features['shortest_word_host'] = len(min(host_words, key=len, default=''))
    features['shortest_word_path'] = len(min(path_words, key=len, default=''))
    features['longest_word_host'] = len(max(host_words, key=len, default=''))
    features['longest_word_path'] = len(max(path_words, key=len, default=''))

    features['shortest_words_raw'] = len(min(all_words, key=len, default=''))
    features['longest_words_raw'] = len(max(all_words, key=len, default=''))
    features['avg_word_host'] = np.mean([len(word) for word in host_words]) if host_words else 0
    features['avg_word_path'] = np.mean([len(word) for word in path_words]) if path_words else 0
    features['avg_words_raw'] = np.mean([len(word) for word in all_words]) if all_words else 0

    features['phish_hints'] = int(any(hint in url for hint in ['secure', 'login', 'update', 'verify']))

    features['domain_in_brand'] = int(domain_info.domain in url)
    features['brand_in_subdomain'] = int(any(brand in parsed_url.hostname for brand in ['brand', 'company'])) if parsed_url.hostname else 0
    features['brand_in_path'] = int(any(brand in parsed_url.path for brand in ['brand', 'company']))

    suspicious_tlds = ['.tk', '.ml', '.ga', '.cf', '.gq']
    features['suspecious_tld'] = int(any(tld in parsed_url.hostname for tld in suspicious_tlds)) if parsed_url.hostname else 0

    # Placeholder for Int Errors and Ext Errors
    features['ratio_intErrors'] = 0  # Placeholder, logic needed to calculate
    features['ratio_extErrors'] = 0  # Placeholder, logic needed to calculate
    features['submit_email'] = int(bool(soup.find('input', {'type': 'email'}))) if 'soup' in locals() else 0

    features['whois_registered_domain'] = 0
    features['domain_registration_length'] = -1

    # Attempt to get WHOIS info directly if needed
    try:
        whois_info = whois.whois(parsed_url.hostname)
        features['whois_registered_domain'] = int(bool(whois_info.domain_name))
        if whois_info.creation_date and whois_info.expiration_date:
            if isinstance(whois_info.creation_date, list):
                creation_date = whois_info.creation_date[0]
            else:
                creation_date = whois_info.creation_date

            if isinstance(whois_info.expiration_date, list):
                expiration_date = whois_info.expiration_date[0]
            else:
                expiration_date = whois_info.expiration_date

            features['domain_registration_length'] = (expiration_date - creation_date).days
        else:
            features['domain_registration_length'] = -1
    except Exception as e:
        print(f"Error fetching WHOIS info: {e}")

    return features

# Load and preprocess data
data_path = '/content/drive/MyDrive/dataset_phishing.csv'  # Adjust this to your data path
df = pd.read_csv(data_path, encoding='utf-8')
df.dropna(inplace=True)

# Map the 'status' column to numeric values before splitting the data
df['status'] = df['status'].map({'legitimate': 0, 'phishing': 1})

# Feature and target selection
feature_columns = [
    'url', 'length_url', 'length_hostname',  # Added 'url' as a feature temporarily
    'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq',
    'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon',
    'nb_comma', 'nb_semicolumn', 'nb_dollar', 'nb_space', 'nb_www', 'nb_com',
    'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url', 'ratio_digits_host',
    'punycode', 'port', 'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain',
    'nb_subdomains', 'prefix_suffix', 'random_domain', 'shortening_service',
    'path_extension', 'nb_redirection', 'nb_external_redirection', 'length_words_raw',
    'char_repeat', 'shortest_words_raw', 'shortest_word_host', 'shortest_word_path',
    'longest_words_raw', 'longest_word_host', 'longest_word_path', 'avg_words_raw',
    'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand',
    'brand_in_subdomain', 'brand_in_path', 'suspecious_tld', 'nb_hyperlinks',
    'ratio_intHyperlinks', 'ratio_extHyperlinks', 'ratio_nullHyperlinks',
    'ratio_intRedirection', 'ratio_extRedirection', 'ratio_intErrors', 'ratio_extErrors',
    'external_favicon', 'links_in_tags', 'submit_email',
    'ratio_intMedia', 'ratio_extMedia', 'sfh', 'iframe', 'popup_window',
    'safe_anchor', 'onmouseover', 'right_clic', 'empty_title', 'domain_in_title',
    'domain_with_copyright', 'whois_registered_domain', 'domain_registration_length'
]

# Selecting features without including 'url' for model training
X = df[feature_columns].drop(columns=['url']).values  # Exclude the 'url' column
y = df['status'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

# Hyperparameters
input_size = X_train.shape[1]
hidden_size = 128
num_layers = 2
output_size = 1
num_epochs = 100
learning_rate = 0.005

# Initialize the model, loss function, and optimizer
model = RNNModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train.unsqueeze(1))  # Reshape for RNN
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    y_pred_train = model(X_train.unsqueeze(1)).round()
    y_pred_test = model(X_test.unsqueeze(1)).round()

    train_accuracy = accuracy_score(y_train, y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    print(f'Training Accuracy: {train_accuracy:.4f}')
    print(f'Test Accuracy: {test_accuracy:.4f}')

    print('Classification Report:')
    print(classification_report(y_test, y_pred_test))

    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred_test))

# Predicting new URL
urls = [
    'https://now-get-free-verified.vercel.app/',
    'https://s.monsoondealz.xyz/',
    'http://dpd-hr.receiving-delivery.com/track/5387861827/',
    'https://www.google.com/',
    'https://www.facebook.com/',
    'http://rgipt.ac.in',
    'https://www.youtube.com/',
]

# Extract features for each URL
feature_values_list = []
for url in urls:
    new_features = extract_features(url)

    # Extract feature values into a list
    feature_values = [new_features[col] for col in feature_columns if col != 'url']

    # Append to the list
    feature_values_list.append(feature_values)

# Create DataFrame for the new features
new_features_df = pd.DataFrame(feature_values_list, columns=[col for col in feature_columns if col != 'url'])

# Scale the features
new_features_scaled = scaler.transform(new_features_df)

# Convert to PyTorch tensor
new_features_tensor = torch.tensor(new_features_scaled, dtype=torch.float32).unsqueeze(1)

# Make predictions
model.eval()
with torch.no_grad():
    predictions = model(new_features_tensor).round()

# Print predictions for each URL
for url, pred in zip(urls, predictions):
    print(f'Prediction for {url}: {"Phishing" if pred.item() == 1 else "Legitimate"}')

# Print extracted feature values list for debugging
for url, features in zip(urls, feature_values_list):
    print(f'Extracted feature values for {url}: {features}')

# Save the model and scaler
model_path = '/content/drive/MyDrive/sweeb11.pth'
scaler_path = '/content/drive/MyDrive/sweb12.pkl'

# Save the model
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

# Save the scaler
joblib.dump(scaler, scaler_path)
print(f"Scaler saved to {scaler_path}")

# Confirm the files exist
import os
print("Files in drive:")
print(os.listdir('/content/drive/MyDrive'))

Epoch [10/100], Loss: 0.3018
Epoch [20/100], Loss: 0.2615
Epoch [30/100], Loss: 0.2473
Epoch [40/100], Loss: 0.2349
Epoch [50/100], Loss: 0.2190
Epoch [60/100], Loss: 0.1992
Epoch [70/100], Loss: 0.1759
Epoch [80/100], Loss: 0.1502
Epoch [90/100], Loss: 0.1308
Epoch [100/100], Loss: 0.1147
Training Accuracy: 0.9611
Test Accuracy: 0.9353
Classification Report:
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94      1157
         1.0       0.95      0.92      0.93      1129

    accuracy                           0.94      2286
   macro avg       0.94      0.94      0.94      2286
weighted avg       0.94      0.94      0.94      2286

Confusion Matrix:
[[1098   59]
 [  89 1040]]
Error fetching URL content: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'
Error fetching URL content: HTTPSConnectionPool(host='s.monsoondealz.xyz', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/dataset_phishing.csv'
df = pd.read_csv(data_path, encoding='utf-8')
# Load and preprocess data
# Load and preprocess data
df.dropna(inplace=True)

# Features and target
# Exclude 'url' from features as it's non-numerical
features = [
   'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq',
    'nb_underscore', 'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma', 'nb_semicolumn',
    'nb_dollar', 'nb_space', 'nb_www', 'nb_com', 'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url',
    'ratio_digits_host', 'punycode', 'port', 'tld_in_path', 'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains',
    'prefix_suffix', 'random_domain', 'shortening_service', 'path_extension', 'nb_redirection', 'nb_external_redirection',
    'length_words_raw', 'char_repeat', 'shortest_words_raw', 'shortest_word_host', 'shortest_word_path',
    'longest_words_raw', 'longest_word_host', 'longest_word_path', 'avg_words_raw', 'avg_word_host', 'avg_word_path',
    'phish_hints', 'domain_in_brand', 'brand_in_subdomain', 'brand_in_path', 'suspecious_tld',
    'nb_hyperlinks', 'ratio_intHyperlinks', 'ratio_extHyperlinks', 'ratio_nullHyperlinks', 'nb_extCSS',
    'ratio_intRedirection', 'ratio_extRedirection', 'ratio_intErrors', 'ratio_extErrors', 'login_form',
    'external_favicon', 'links_in_tags', 'submit_email', 'ratio_intMedia', 'ratio_extMedia', 'sfh', 'iframe',
    'popup_window', 'safe_anchor', 'onmouseover', 'right_clic', 'empty_title', 'domain_in_title',
    'domain_with_copyright', 'whois_registered_domain', 'domain_registration_length', 'domain_age'
]


df['status'] = df['status'].map({'phishing': 1, 'legitimate': 0})

X = df[features] # Exclude the 'url' column
y = df['status']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Define the RNN model
class PhishingRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2):
        super(PhishingRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# Hyperparameters
input_size = X_train_tensor.shape[1]
hidden_size = 64
output_size = 2
num_layers = 2
num_epochs = 100
batch_size = 64
learning_rate = 0.005

# Initialize the model, loss function, and optimizer
model = PhishingRNN(input_size, hidden_size, output_size, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train_tensor.unsqueeze(1))
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluating the model
model.eval()
with torch.no_grad():
    y_pred_train = model(X_train_tensor.unsqueeze(1)).argmax(dim=1)
    y_pred_test = model(X_test_tensor.unsqueeze(1)).argmax(dim=1)

    train_accuracy = accuracy_score(y_train, y_pred_train.numpy())
    test_accuracy = accuracy_score(y_test, y_pred_test.numpy())

    print(f'Train Accuracy: {train_accuracy:.4f}')
    print(f'Test Accuracy: {test_accuracy:.4f}')
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test.numpy()))
    print("Classification Report:\n", classification_report(y_test, y_pred_test.numpy()))

# Function to test a new URL
# Function to test a new URL
def test_new_url(url_features):
    # Ensure url_features has the correct number of features (87 in this case)
    assert len(url_features) == len(features), f"Expected {len(features)} features, but got {len(url_features)}."

    model.eval()
    url_features_scaled = scaler.transform([url_features])  # Scale the features
    url_tensor = torch.tensor(url_features_scaled, dtype=torch.float32).unsqueeze(1)
    with torch.no_grad():
        prediction = model(url_tensor).argmax(dim=1).item()
    return 'Phishing' if prediction == 1 else 'Legitimate'

new_url_features = [
  23, 15, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 1, 1, False, True, 0.0, False, False, False, False, False, -1, False, True, 0, 0.0, 0, 1, 0, 0.0, 14, 0.5714285714285714, 0.42857142857142855, 0.0, 0, 0, 0, 0, 0, 27, 0, 5, 1, 0, 0, 1, 0, 0, 0, 0, 0, 15, 6, 3, 0, 7, 0, 3, 7, 4.333333333333333, 0, 4.333333333333333, 0, 1, 0, 0, 0, 0, 0, 0, 1, 7305, 7119
]
# Example function to classify a new URL
def test_new_url(url_features):
    model.eval()
    url_features_scaled = scaler.transform([url_features])  # Scale the features
    url_tensor = torch.tensor(url_features_scaled, dtype=torch.float32).unsqueeze(1)
    with torch.no_grad():
        prediction = model(url_tensor).argmax(dim=1).item()
    return 'Phishing' if prediction == 1 else 'Legitimate'

# Test the new URL
result = test_new_url(new_url_features)
print(f'The new URL is classified as: {result}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch [10/100], Loss: 0.3151
Epoch [20/100], Loss: 0.2445
Epoch [30/100], Loss: 0.2258
Epoch [40/100], Loss: 0.2103
Epoch [50/100], Loss: 0.1932
Epoch [60/100], Loss: 0.1725
Epoch [70/100], Loss: 0.1515
Epoch [80/100], Loss: 0.1318
Epoch [90/100], Loss: 0.1124
Epoch [100/100], Loss: 0.0962
Train Accuracy: 0.9662
Test Accuracy: 0.9377
Confusion Matrix:
 [[1346   76]
 [ 102 1334]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.95      0.94      1422
           1       0.95      0.93      0.94      1436

    accuracy                           0.94      2858
   macro avg       0.94      0.94      0.94      2858
weighted avg       0.94      0.94      0.94      2858

The new URL is classified as: Phishing


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
pip install requests beautifulsoup4 tldextract python-whois

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import re
import tldextract
import whois
import datetime
import numpy as np
import socket

# Feature extraction function
def is_valid_ip(ip):
    """Check if the hostname is an IP address."""
    try:
        socket.inet_aton(ip)
        return True
    except socket.error:
        return False

def extract_features(url):
    features = {}

    # Basic URL parsing
    parsed_url = urlparse(url)
    domain_info = tldextract.extract(url)

    # Basic URL Length Features
    features['length_url'] = len(url)
    features['length_hostname'] = len(parsed_url.hostname or '')

    # Token Counts
    tokens = ['.', '-', '@', '?', '&', '|', '=', '_', '~', '%', '/', '*', ':', ',', ';', '$', ' ']
    token_names = ['nb_dots', 'nb_hyphens', 'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore',
                   'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma',
                   'nb_semicolumn', 'nb_dollar', 'nb_space']

    for token, name in zip(tokens, token_names):
        features[name] = url.count(token)

    features['nb_www'] = url.count('www')
    features['nb_com'] = url.count('.com')
    features['nb_dslash'] = url.count('//')

    # Path Features
    features['http_in_path'] = int('http' in parsed_url.path)
    features['https_token'] = int('https' in url)
    features['ratio_digits_url'] = sum(c.isdigit() for c in url) / len(url) if len(url) > 0 else 0

    path_extensions = ['.php', '.asp', '.aspx', '.jsp']
    features['path_extension'] = int(any(ext in parsed_url.path for ext in path_extensions))
    features['prefix_suffix'] = int(any(part in url for part in ['.htm', '.html']))
    random_strings = ['random', '1234', 'xyz']
    features['random_domain'] = int(any(sub in parsed_url.hostname for sub in random_strings)) if parsed_url.hostname else 0

    # Domain Features
    features['punycode'] = int(parsed_url.hostname.encode('idna').decode('utf-8') != parsed_url.hostname) if parsed_url.hostname else 0
    shortening_services = ['bit.ly', 'tinyurl.com', 'goo.gl']
    features['shortening_service'] = int(any(service in url for service in shortening_services))
    features['port'] = parsed_url.port if parsed_url.port else -1
    tlds = ['.com', '.net', '.org']
    features['tld_in_path'] = int(any(tld in parsed_url.path for tld in tlds))
    features['tld_in_subdomain'] = int(domain_info.suffix in parsed_url.hostname) if parsed_url.hostname else 0

    # Check if IP address is used as hostname
    features['ip'] = int(is_valid_ip(parsed_url.hostname)) if parsed_url.hostname else 0

    # Ratio of digits in hostname
    features['ratio_digits_host'] = sum(c.isdigit() for c in parsed_url.hostname) / len(parsed_url.hostname) if parsed_url.hostname and len(parsed_url.hostname) > 0 else 0

    # Abnormal subdomain
    features['abnormal_subdomain'] = int(len(domain_info.subdomain.split('.')) > 2) if domain_info.subdomain else 0

    # Number of subdomains
    features['nb_subdomains'] = len(domain_info.subdomain.split('.')) if domain_info.subdomain else 0

    # Initialize media ratios
    features['ratio_intMedia'] = 0
    features['ratio_extMedia'] = 0

    # Default values for URL content-based features
    default_features = {
        'nb_hyperlinks': 0,
        'ratio_extHyperlinks': 0,
        'ratio_intHyperlinks': 0,
        'ratio_nullHyperlinks': 0,
        'nb_redirection': 0,
        'nb_external_redirection': 0,
        'ratio_intRedirection': 0,
        'ratio_extRedirection': 0,
        'links_in_tags': 0,
        'external_favicon': 0,
        'iframe': 0,
        'popup_window': 0,
        'sfh': 0,
        'safe_anchor': 0,
        'onmouseover': 0,
        'right_clic': 0,
        'empty_title': 0,
        'domain_in_title': 0,
        'domain_with_copyright': 0
    }

    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Count Links in the Page
        links = soup.find_all('a')
        total_links = len(links)
        features['nb_hyperlinks'] = total_links

        if total_links > 0:
            ext_links = len([link for link in links if link.get('href', '').startswith('http')])
            int_links = len([link for link in links if not link.get('href', '').startswith('http')])
            null_links = len([link for link in links if not link.get('href')])

            features['ratio_extHyperlinks'] = ext_links / total_links
            features['ratio_intHyperlinks'] = int_links / total_links
            features['ratio_nullHyperlinks'] = null_links / total_links
        else:
            features['ratio_extHyperlinks'] = 0
            features['ratio_intHyperlinks'] = 0
            features['ratio_nullHyperlinks'] = 0

        # Calculate redirections
        features['nb_redirection'] = len(response.history)
        features['nb_external_redirection'] = len([resp for resp in response.history if urlparse(resp.url).netloc != parsed_url.netloc])

        # Internal and external redirection ratios
        features['ratio_intRedirection'] = len([resp for resp in response.history if urlparse(resp.url).netloc == parsed_url.netloc]) / len(response.history) if response.history else 0
        features['ratio_extRedirection'] = features['nb_external_redirection'] / len(response.history) if response.history else 0

        # Check for favicon
        favicon_tag = soup.find('link', rel='shortcut icon')
        features['external_favicon'] = int(favicon_tag and parsed_url.netloc not in favicon_tag.get('href', ''))

        # Links in specific tags
        script_links = len(soup.find_all('script', src=True))
        link_links = len(soup.find_all('link', href=True))
        features['links_in_tags'] = script_links + link_links

        media_links = soup.find_all(['img', 'video', 'audio'])
        features['ratio_extMedia'] = len(media_links) / len(links) if len(links) > 0 else 0
        features['iframe'] = int(bool(soup.find('iframe')))
        features['popup_window'] = int(bool(soup.find('script', string=re.compile('window.open'))))

        safe_anchors = [link.get('rel') != 'nofollow' for link in links]
        features['sfh'] = int(any(form.get('action', '').startswith('http') for form in soup.find_all('form')))
        features['safe_anchor'] = int(all(safe_anchors))

        features['onmouseover'] = int(bool(soup.find_all(attrs={"onmouseover": True})))
        features['right_clic'] = int(bool(soup.find_all(attrs={"oncontextmenu": True})))
        features['empty_title'] = int(bool(soup.find('title') and not soup.find('title').text.strip()))
        features['domain_in_title'] = int(domain_info.domain in (soup.find('title').text if soup.find('title') else ''))
        features['domain_with_copyright'] = int('copyright' in (soup.find('body').text if soup.find('body') else '').lower())

    except Exception as e:
        print(f"Error fetching URL content: {e}")
        features.update(default_features)

    # Calculate the lengths of words
    path_words = re.findall(r'\w+', parsed_url.path)
    host_words = re.findall(r'\w+', parsed_url.hostname) if parsed_url.hostname else []
    all_words = path_words + host_words
    features['length_words_raw'] = len(' '.join(all_words))

    features['char_repeat'] = len({char: url.count(char) for char in set(url) if url.count(char) > 1})

    # Capture lengths of words
    features['shortest_word_host'] = len(min(host_words, key=len, default=''))
    features['shortest_word_path'] = len(min(path_words, key=len, default=''))
    features['longest_word_host'] = len(max(host_words, key=len, default=''))
    features['longest_word_path'] = len(max(path_words, key=len, default=''))

    features['shortest_words_raw'] = len(min(all_words, key=len, default=''))
    features['longest_words_raw'] = len(max(all_words, key=len, default=''))
    features['avg_word_host'] = np.mean([len(word) for word in host_words]) if host_words else 0
    features['avg_word_path'] = np.mean([len(word) for word in path_words]) if path_words else 0
    features['avg_words_raw'] = np.mean([len(word) for word in all_words]) if all_words else 0

    features['phish_hints'] = int(any(hint in url for hint in ['secure', 'login', 'update', 'verify']))

    features['domain_in_brand'] = int(domain_info.domain in url)
    features['brand_in_subdomain'] = int(any(brand in parsed_url.hostname for brand in ['brand', 'company'])) if parsed_url.hostname else 0
    features['brand_in_path'] = int(any(brand in parsed_url.path for brand in ['brand', 'company']))

    suspicious_tlds = ['.tk', '.ml', '.ga', '.cf', '.gq']
    features['suspecious_tld'] = int(any(tld in parsed_url.hostname for tld in suspicious_tlds)) if parsed_url.hostname else 0

    # Placeholder for Int Errors and Ext Errors
    features['ratio_intErrors'] = 0  # Placeholder, logic needed to calculate
    features['ratio_extErrors'] = 0  # Placeholder, logic needed to calculate
    features['submit_email'] = int(bool(soup.find('input', {'type': 'email'}))) if 'soup' in locals() else 0

    try:
        whois_info = whois.whois(parsed_url.hostname)
        features['whois_registered_domain'] = int(bool(whois_info.domain_name))
        if whois_info.creation_date and whois_info.expiration_date:
            if isinstance(whois_info.creation_date, list):
                creation_date = whois_info.creation_date[0]
            else:
                creation_date = whois_info.creation_date

            if isinstance(whois_info.expiration_date, list):
                expiration_date = whois_info.expiration_date[0]
            else:
                expiration_date = whois_info.expiration_date

            features['domain_registration_length'] = (expiration_date - creation_date).days
            features['domain_age'] = (datetime.datetime.now() - creation_date).days
        else:
            features['domain_registration_length'] = -1
            features['domain_age'] = -1
    except Exception as e:
        print(f"Error fetching WHOIS info: {e}")
        features['whois_registered_domain'] = 0
        features['domain_registration_length'] = -1
        features['domain_age'] = -1

    return features

# Example usage
url = 'https://www.facebook.com'
extracted_features = extract_features(url)

# Print features in list format including the URL and feature names
feature_names = ['URL'] + list(extracted_features.keys())
feature_values = [url] + list(extracted_features.values())
feature_list = list(zip(feature_names, feature_values))
feature_lists = [url] + [value for value in extracted_features.values()]
print("f", feature_list)
print("ff", feature_lists)

Error fetching URL content: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'
f [('URL', 'https://www.facebook.com'), ('length_url', 24), ('length_hostname', 16), ('nb_dots', 2), ('nb_hyphens', 0), ('nb_at', 0), ('nb_qm', 0), ('nb_and', 0), ('nb_or', 0), ('nb_eq', 0), ('nb_underscore', 0), ('nb_tilde', 0), ('nb_percent', 0), ('nb_slash', 2), ('nb_star', 0), ('nb_colon', 1), ('nb_comma', 0), ('nb_semicolumn', 0), ('nb_dollar', 0), ('nb_space', 0), ('nb_www', 1), ('nb_com', 1), ('nb_dslash', 1), ('http_in_path', 0), ('https_token', 1), ('ratio_digits_url', 0.0), ('path_extension', 0), ('prefix_suffix', 0), ('random_domain', 0), ('punycode', 0), ('shortening_service', 0), ('port', -1), ('tld_in_path', 0), ('tld_in_subdomain', 1), ('ip', 0), ('ratio_digits_host', 0.0), ('abnormal_subdomain', 0), ('nb_subdomains', 1), ('ratio_intMedia', 0), ('ratio_extMedia', 0), ('nb_hyperlinks', 0), ('ratio_extHyperlinks', 0), ('ratio_intHyperlinks', 0), ('ratio_nullHyp

In [ ]:
import re
import pandas as pd
from urllib.parse import urlparse
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# URL Feature Extraction
def fetch_url_content(url):
    """Fetch URL content. Dummy implementation for illustration."""
    try:
        response = requests.get(url)
        return response.text
    except Exception as e:
        print(f"Error fetching URL content: {e}")
        return None

def extract_features(url):
    """Extract features from URL for phishing detection."""
    features = {}

    # Basic Length-based Features
    features['url_length'] = len(url)
    features['hostname_length'] = len(urlparse(url).hostname or '')
    features['path_length'] = len(urlparse(url).path or '')

    # Token-based Features
    features['num_tokens'] = len(url.split('/'))

    # Presence of Special Characters
    features['num_dots'] = url.count('.')
    features['num_hyphens'] = url.count('-')
    features['num_ip'] = int(bool(re.search(r'\d+\.\d+\.\d+\.\d+', url)))

    # Fetch URL content and calculate content-based features
    try:
        content = fetch_url_content(url)
        if content:
            num_words = len(content.split())
            features['num_words'] = num_words
            features['longest_word'] = len(max(content.split(), key=len, default=''))
        else:
            features['num_words'] = 0
            features['longest_word'] = 0
    except Exception as e:
        print(f"Error fetching URL content: {e}")
        features['num_words'] = 0
        features['longest_word'] = 0

    return features

# Load and Prepare Data
def load_and_prepare_data(filepath):
    """Load dataset and prepare for training."""
    data = pd.read_csv(filepath)
    X = data['url'].apply(extract_features).tolist()
    y = data['label']

    # Convert features to a format suitable for machine learning
    vec = DictVectorizer()
    X_vectorized = vec.fit_transform(X)

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

    # Standardize features
    scaler = StandardScaler(with_mean=False)
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test

# Define the Neural Network Model
class PhishingModel(nn.Module):
    def __init__(self, input_dim):
        super(PhishingModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # Binary classification

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Main Execution
def main():
    # Load and prepare data
    X_train_scaled, X_test_scaled, y_train, y_test = load_and_prepare_data('/content/drive/MyDrive/dataset_phishing.csv')

    # Convert data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_scaled.toarray(), dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test_scaled.toarray(), dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    # Initialize the model, loss function, and optimizer
    input_dim = X_train_scaled.shape[1]
    model = PhishingModel(input_dim)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    epochs = 100
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(X_train_tensor)
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}')

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == y_test_tensor).sum().item()
        accuracy = correct / len(y_test_tensor)
        print(f'Test Accuracy: {accuracy:.4f}')

if __name__ == '__main__':
    main()


KeyError: "['label'] not found in axis"

In [ ]:
pip install tensorflow

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from urllib.parse import urlparse
import tldextract
import re

# Function to extract features from a URL
def extract_features(url):
    # Parse the URL
    parsed_url = urlparse(url)
    ext = tldextract.extract(url)

    # Extract features
    features = {
        'having_IP': 1 if re.match(r"(\d{1,3}\.){3}\d{1,3}", parsed_url.netloc) else 0,
        'URL_Length': len(url),
        'Shortining_Service': 1 if any(shortener in url for shortener in ["bit.ly", "tinyurl.com", "goo.gl", "ow.ly", "t.co"]) else 0,
        'having_At_Symbol': 1 if '@' in url else 0,
        'double_slash_redirecting': 1 if '//' in parsed_url.path else 0,
        'Prefix_Suffix': 1 if '-' in ext.domain else 0,
        'having_Sub_Domain': 1 if ext.subdomain else 0,
        'SSLfinal_State': 1 if parsed_url.scheme == 'https' else 0,
        'Domain_registeration_length': 0, # Placeholder, you need to implement this feature
        'Favicon': 0, # Placeholder, you need to implement this feature
        'port': parsed_url.port if parsed_url.port else 0,
        'HTTPS_token': 1 if 'https' in parsed_url.netloc.lower() else 0,
        'Request_URL': 0, # Placeholder, you need to implement this feature
        'URL_of_Anchor': 0, # Placeholder, you need to implement this feature
        'Links_in_tags': 0, # Placeholder, you need to implement this feature
        'SFH': 0, # Placeholder, you need to implement this feature
        'Submitting_to_email': 0, # Placeholder, you need to implement this feature
        'Abnormal_URL': 0, # Placeholder, you need to implement this feature
        'Redirect': 0, # Placeholder, you need to implement this feature
        'on_mouseover': 0, # Placeholder, you need to implement this feature
        'RightClick': 0, # Placeholder, you need to implement this feature
        'popUpWidnow': 0, # Placeholder, you need to implement this feature
        'Iframe': 0, # Placeholder, you need to implement this feature
        'age_of_domain': 0, # Placeholder, you need to implement this feature
        'DNSRecord': 0, # Placeholder, you need to implement this feature
        'web_traffic': 0, # Placeholder, you need to implement this feature
        'Page_Rank': 0, # Placeholder, you need to implement this feature
        'Google_Index': 0, # Placeholder, you need to implement this feature
        'Links_pointing_to_page': 0, # Placeholder, you need to implement this feature
        'Statistical_report': 0, # Placeholder, you need to implement this feature
        'domain_length': len(ext.domain),
        'subdomain_length': len(ext.subdomain),
        'suffix_length': len(ext.suffix),
        'path_length': len(parsed_url.path),
        'query_length': len(parsed_url.query),
        'has_https': 1 if parsed_url.scheme == 'https' else 0,
        'num_digits_in_domain': len(re.findall(r'\d', ext.domain)),
        'num_subdomains': ext.subdomain.count('.') + 1 if ext.subdomain else 0,
        'num_special_chars': len(re.findall(r'\W', url)),
    }

    return pd.DataFrame([features]) # Return a DataFrame with all 30 features

# Load dataset and prepare the data
data = pd.read_csv('/content/drive/MyDrive/phishing.csv')

# Extract target
X = data.copy()
y = X.pop('Target')

# Drop unwanted columns
X = data.drop(columns=['index','Target'])

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, train_size=0.75)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.float32).unsqueeze(1)

# Create DataLoader for batches
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=512, shuffle=False)

# Define the PyTorch model
class PhishingDetectionModel(nn.Module):
    def __init__(self, input_dim):
        super(PhishingDetectionModel, self).__init__()
        self.model = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Initialize the model, loss function, and optimizer
model = PhishingDetectionModel(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X_batch.size(0)
    return running_loss / len(train_loader.dataset)

# Validation function
def validate(model, valid_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in valid_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            running_loss += loss.item() * X_batch.size(0)
            preds = outputs.round()
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)
    accuracy = correct / total
    return running_loss / len(valid_loader.dataset), accuracy

# Training loop
n_epochs = 200
early_stopping_patience = 20
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(n_epochs):
    train_loss = train(model, train_loader, criterion, optimizer)
    val_loss, val_accuracy = validate(model, valid_loader, criterion)
    print(f'Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    # Early stopping logic
    if val_loss < best_val_loss - 0.01:
        best_val_loss = val_loss
        patience_counter = 0
        best_model_state = model.state_dict()
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

# Load the best model weights
model.load_state_dict(best_model_state)

# Function to predict if a URL is suspicious
def predict_url(url):
    features = extract_features(url)
    features = scaler.transform(features)  # Scale the features
    features_tensor = torch.tensor(features, dtype=torch.float32)
    model.eval()
    with torch.no_grad():
        prediction = model(features_tensor)
        return "Suspicious" if prediction.item() >= 0.5 else "Not Suspicious"

# Example usage: Predicting a URL
url_to_test = "http://example.com"
result = predict_url(url_to_test)
print(f"The URL {url_to_test} is {result}.")

Epoch 1/200, Train Loss: 0.2485, Val Loss: 0.1801, Val Accuracy: 0.9338
Epoch 2/200, Train Loss: 0.1649, Val Loss: 0.1487, Val Accuracy: 0.9399
Epoch 3/200, Train Loss: 0.1472, Val Loss: 0.1385, Val Accuracy: 0.9468
Epoch 4/200, Train Loss: 0.1354, Val Loss: 0.1256, Val Accuracy: 0.9472
Epoch 5/200, Train Loss: 0.1251, Val Loss: 0.1148, Val Accuracy: 0.9555
Epoch 6/200, Train Loss: 0.1172, Val Loss: 0.1153, Val Accuracy: 0.9533
Epoch 7/200, Train Loss: 0.1064, Val Loss: 0.1048, Val Accuracy: 0.9551
Epoch 8/200, Train Loss: 0.1036, Val Loss: 0.1022, Val Accuracy: 0.9573
Epoch 9/200, Train Loss: 0.1036, Val Loss: 0.0999, Val Accuracy: 0.9602
Epoch 10/200, Train Loss: 0.0970, Val Loss: 0.0960, Val Accuracy: 0.9606
Epoch 11/200, Train Loss: 0.0917, Val Loss: 0.0979, Val Accuracy: 0.9569
Epoch 12/200, Train Loss: 0.0879, Val Loss: 0.0914, Val Accuracy: 0.9635
Epoch 13/200, Train Loss: 0.0900, Val Loss: 0.0924, Val Accuracy: 0.9627
Epoch 14/200, Train Loss: 0.0859, Val Loss: 0.0889, Val Accu

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- URL_Length
- domain_length
- has_https
- having_IP
- num_digits_in_domain
- ...
Feature names seen at fit time, yet now missing:
- URLURL_Length
- having_IPhaving_IP_Address


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Use the output from the last time step
        return out

# Load dataset
data = pd.read_csv('/content/drive/MyDrive/phishing.csv')

# Convert categorical features to numerical values
label_encoders = {}
for column in ['UsingIP', 'LongURL', 'ShortURL', 'Symbol@', 'Redirecting//', 'PrefixSuffix-', 'SubDomains', 'HTTPS', 'Favicon', 'NonStdPort', 'HTTPSDomainURL', 'RequestURL', 'AnchorURL', 'LinksInScriptTags', 'ServerFormHandler', 'InfoEmail', 'AbnormalURL', 'WebsiteForwarding', 'StatusBarCust', 'DisableRightClick', 'UsingPopupWindow', 'IframeRedirection']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Define features and target variable
features = ['UsingIP', 'LongURL', 'ShortURL', 'Symbol@', 'Redirecting//', 'PrefixSuffix-', 'SubDomains', 'HTTPS', 'DomainRegLen', 'Favicon', 'NonStdPort', 'HTTPSDomainURL', 'RequestURL', 'AnchorURL', 'LinksInScriptTags', 'ServerFormHandler', 'InfoEmail', 'AbnormalURL', 'WebsiteForwarding', 'StatusBarCust', 'DisableRightClick', 'UsingPopupWindow', 'IframeRedirection', 'AgeofDomain', 'DNSRecording', 'WebsiteTraffic', 'PageRank', 'GoogleIndex', 'LinksPointingToPage', 'StatsReport']
X = data[features].values
y = data['class'].values  # Assuming 'class' is your target variable

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
class URLDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

train_dataset = URLDataset(X_train, y_train)
test_dataset = URLDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the RNN model
input_size = len(features)
hidden_size = 128
output_size = len(np.unique(y))
model = RNNModel(input_size, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features.unsqueeze(1))  # Add batch dimension
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in test_loader:
            outputs = model(features.unsqueeze(1))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {100 * correct / total}%')

# Save the model
torch.save(model.state_dict(), 'rnn_model.pth')

# Load the model for prediction
model = RNNModel(input_size, hidden_size, output_size)
model.load_state_dict(torch.load('rnn_model.pth'))
model.eval()

# Load sample data from CSV
sample_data = pd.read_csv('/content/drive/MyDrive/phishing.csv')

# Convert categorical features to numerical values using the same encoders
for column, le in label_encoders.items():
    sample_data[column] = le.transform(sample_data[column])

# Normalize sample data using the same scaler
X_sample = scaler.transform(sample_data[features])

# Convert sample data to PyTorch tensor
X_sample_tensor = torch.tensor(X_sample, dtype=torch.float32).unsqueeze(1)  # Add batch dimension

# Make prediction
with torch.no_grad():
    outputs = model(X_sample_tensor)
    _, predicted = torch.max(outputs, 1)

print(f'Predicted classes: {predicted.numpy()}')

IndexError: Target -1 is out of bounds.

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report
from torch.utils.data import DataLoader, TensorDataset

# Load data
data = pd.read_csv("/content/drive/MyDrive/phishing.csv")
data = data.drop(['Index'], axis=1)

# Splitting the dataset into dependent and independent features
X = data.drop(["class"], axis=1)
y = data["class"]

y = (y == 1).astype(int)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

# DataLoader
batch_size = 64
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define the Fully Connected Neural Network model
class PhishingFCNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PhishingFCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

# Model instantiation
input_size = X_train.shape[1]  # This should be the number of features
hidden_size = 128
output_size = 1

model = PhishingFCNN(input_size, hidden_size, output_size)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 20
model.train()

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        labels = labels.unsqueeze(1)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test).round().squeeze()

# Convert to numpy arrays
y_test = y_test.numpy()
y_pred = y_pred.numpy()

# Metrics calculation
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {acc:.3f}')
print(f'F1 Score: {f1:.3f}')
print(f'Recall: {rec:.3f}')
print(f'Precision: {prec:.3f}')
print(f'Classification Report:\n{report}')

# Sample input for testing
# Make sure the number of features matches the input_size of the model
sample_input = np.array([[1, 0, 0, 1, 1, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1]])  # Adjust the size here
sample_input_tensor = torch.tensor(sample_input, dtype=torch.float32)

# Prediction
model.eval()
with torch.no_grad():
    sample_prediction = model(sample_input_tensor).item()
    predicted_class = 1 if sample_prediction >= 0.5 else 0

print(f'Sample input prediction: {predicted_class} (Probability: {sample_prediction:.4f})')

Epoch [1/20], Loss: 0.2912
Epoch [2/20], Loss: 0.0463
Epoch [3/20], Loss: 0.2567
Epoch [4/20], Loss: 0.3615
Epoch [5/20], Loss: 0.0453
Epoch [6/20], Loss: 0.3764
Epoch [7/20], Loss: 0.1466
Epoch [8/20], Loss: 0.0597
Epoch [9/20], Loss: 0.2109
Epoch [10/20], Loss: 0.0437
Epoch [11/20], Loss: 0.2620
Epoch [12/20], Loss: 0.0206
Epoch [13/20], Loss: 0.0925
Epoch [14/20], Loss: 0.1682
Epoch [15/20], Loss: 0.0809
Epoch [16/20], Loss: 0.0433
Epoch [17/20], Loss: 0.0300
Epoch [18/20], Loss: 0.0673
Epoch [19/20], Loss: 0.0601
Epoch [20/20], Loss: 0.1412
Accuracy: 0.956
F1 Score: 0.961
Recall: 0.972
Precision: 0.951
Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.94      0.95       976
         1.0       0.95      0.97      0.96      1235

    accuracy                           0.96      2211
   macro avg       0.96      0.95      0.96      2211
weighted avg       0.96      0.96      0.96      2211

Sample input prediction: 1 (Probabil

In [ ]:
pip install  whois

In [ ]:
import ipaddress
import re
import urllib.request
from bs4 import BeautifulSoup
import socket
import requests
from googlesearch import search
import whois
from datetime import date, datetime
from urllib.parse import urlparse

class FeatureExtraction:
    def __init__(self, url):
        self.url = url
        self.features = []
        self.domain = ""
        self.whois_response = ""
        self.urlparse = ""
        self.response = ""
        self.soup = ""

        try:
            self.response = requests.get(url)
            self.soup = BeautifulSoup(self.response.text, 'html.parser')
        except Exception as e:
            print(f"Error fetching URL: {e}")

        try:
            self.urlparse = urlparse(url)
            self.domain = self.urlparse.netloc
        except Exception as e:
            print(f"Error parsing URL: {e}")

        try:
            self.whois_response = whois.whois(self.domain)
        except Exception as e:
            print(f"Error fetching WHOIS data: {e}")

        # Extracting features
        self.features.append(self.using_ip())
        self.features.append(self.long_url())
        self.features.append(self.short_url())
        self.features.append(self.symbol_in_url())
        self.features.append(self.redirecting())
        self.features.append(self.prefix_suffix())
        self.features.append(self.sub_domains())
        self.features.append(self.https())
        self.features.append(self.domain_registration_length())
        self.features.append(self.favicon())

        # Add more feature extraction methods as necessary
        # ...

    def using_ip(self):
        try:
            ipaddress.ip_address(self.url)
            return -1
        except ValueError:
            return 1

    def long_url(self):
        if len(self.url) < 54:
            return 1
        elif len(self.url) >= 54 and len(self.url) <= 75:
            return 0
        else:
            return -1

    def short_url(self):
        if re.search(r'bit\.ly|goo\.gl|shorte\.st|...|tinyurl\.com', self.url):
            return -1
        return 1

    def symbol_in_url(self):
        if "@" in self.url:
            return -1
        return 1

    def redirecting(self):
        if self.url.rfind('//') > 6:
            return -1
        return 1

    def prefix_suffix(self):
        if '-' in self.domain:
            return -1
        return 1

    def sub_domains(self):
        dot_count = len(re.findall("\.", self.url))
        if dot_count == 1:
            return 1
        elif dot_count == 2:
            return 0
        return -1

    def https(self):
        if self.urlparse.scheme == 'https':
            return 1
        return -1

    def domain_registration_length(self):
        try:
            expiration_date = self.whois_response.expiration_date
            creation_date = self.whois_response.creation_date
            if isinstance(expiration_date, list):
                expiration_date = expiration_date[0]
            if isinstance(creation_date, list):
                creation_date = creation_date[0]

            age = (expiration_date.year - creation_date.year) * 12 + (expiration_date.month - creation_date.month)
            if age >= 12:
                return 1
            return -1
        except Exception:
            return -1

    def favicon(self):
        try:
            for head in self.soup.find_all('head'):
                for link in head.find_all('link', href=True):
                    if self.domain in link['href']:
                        return 1
            return -1
        except Exception:
            return -1

    # Add more methods for other features...

# Example usage:
url = "http://example.com"
fe = FeatureExtraction(url)
print(fe.features)


Error fetching WHOIS data: module 'whois' has no attribute 'whois'
[1, 1, -1, 1, 1, 1, 1, -1, -1, -1]


In [ ]:
import whois
import datetime
from urllib.parse import urlparse
import logging

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def get_whois_info(url):
    try:
        # Extract domain from URL
        parsed_url = urlparse(url)
        domain = parsed_url.netloc or parsed_url.path
        if not domain:
            raise ValueError("Invalid URL format")

        logger.debug(f"Extracted domain: {domain}")

        # Fetch WHOIS information using whois library
        logger.debug("Starting WHOIS lookup")
        w = whois.whois(domain)
        logger.debug("WHOIS lookup completed")

        # Extract domain name, creation date, and expiration date
        domain_name = w.domain_name
        creation_date = w.creation_date
        expiration_date = w.expiration_date

        logger.debug(f"WHOIS data - Domain Name: {domain_name}, Creation Date: {creation_date}, Expiration Date: {expiration_date}")

        # Handle lists and calculate registration length and domain age
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        if isinstance(expiration_date, list):
            expiration_date = expiration_date[0]

        if creation_date and expiration_date:
            registration_length = (expiration_date - creation_date).days
            domain_age = (datetime.datetime.now() - creation_date).days
        else:
            registration_length = -1
            domain_age = -1

        logger.debug(f"Registration Length: {registration_length} days, Domain Age: {domain_age} days")

        return domain_name, creation_date, expiration_date, registration_length, domain_age, None
    except ValueError as e:
        logger.error(f"ValueError: {str(e)}")
        return None, None, None, -1, -1, f"Value error: {str(e)}"
    except Exception as e:
        logger.error(f"Exception occurred: {str(e)}")
        return None, None, None, -1, -1, f"Error fetching WHOIS info: {str(e)}"


def main():
    url = input("Enter the URL: ")
    domain_name, creation_date, expiration_date, registration_length, domain_age, error_message = get_whois_info(url)

    if domain_name:
        print(f"Domain: {domain_name}")
        print(f"Creation Date: {creation_date}")
        print(f"Expiration Date: {expiration_date}")
        print(f"Registration Length (days): {registration_length}")
        print(f"Domain Age (days): {domain_age}")
    else:
        print(f"Error: {error_message}")

if __name__ == "__main__":
    main()


Enter the URL: www.youtube.com
Domain: ['YOUTUBE.COM', 'youtube.com']
Creation Date: 2005-02-15 05:13:12
Expiration Date: 2025-02-15 05:13:12
Registration Length (days): 7305
Domain Age (days): 7151
